In [313]:
from pathlib import Path
import numpy as np
import pandas as pd 
import json
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#from plotly.offline import iplot

# enable plotly in VS Studio Code
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook" 

from workbench.wandb import wandb_model_DB, make_clickable, color_state, get_architecture_from_model_DB_run_id, get_model_DB_run_id_from_architecture, get_wandb_table_as_df


from workbench.utils.utils import convert_to_kb, create_kb_column

import wandb

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [314]:
%reload_ext autoreload
%autoreload

In [315]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [316]:
df  = wandb_model_DB()


In [317]:
df.columns

Index(['id', 'run_name', 'state', 'MACs', 'url', 'architecture', 'FLOPs',
       'alpha', 'classes', 'img_res', 'OPS_mltk', 'channels', 'macs_mltk',
       'variation', 'energy_mltk', 'total_params', 'OPS_INT8_mltk',
       'model_size_kb', 'peak_memory_b', 'macs_INT8_mltk', 'energy_INT8_mltk',
       'trainable_params', 'non_trainable_params', 'tflite_model_size_kb',
       'flash_model_size_b_mltk', 'n_unsupported_layers_mltk',
       'tflite_model_INT8_size_kb', 'flash_model_size_b__INT8_mltk',
       'RAM_runtime_memory_size_b_mltk', 'n_unsupported_layers_INT8_mltk',
       'RAM_runtime_memory_size_b_INT8_mltk', 'opt_RAM_mltk',
       'opt_RAM_INT8_mltk', 'tags', 'created_at', 'path', 'notes',
       'test_accuracy', 'inference_avg_us', 'stm32_inference_ms_INT8'],
      dtype='object')

In [318]:
df["peak_memory_kb"] = df["peak_memory_b"]/1024.0
df["flash_model_size_kb_INT8_mltk"] = df["flash_model_size_b__INT8_mltk"]/1024.0
df["RAM_runtime_memory_size_kb_INT8_mltk"] = df["RAM_runtime_memory_size_b_INT8_mltk"]/1024.0
df["alpha"] = df["alpha"].astype(str)
df["inference_GPU_ms"] = df["inference_avg_us"]/1000.
df.test_accuracy.fillna(0, inplace=True)
df['test_accuracy'] = df['test_accuracy'].replace("NaN", 0)
df.style.format({'url': make_clickable}).applymap(color_state, subset=["state"])

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms
0,t760hyy4,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBN,finished,21.948 M,https://wandb.ai/susbrock/model_DB/runs/t760hyy4,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBN,45.461 M,0.7,2,96,45460785.000000,3,21947885.000000,l5ll32pl32b0.25.MV1NBN,0.017834,162345,45460785.000000,833.953125,152064.000000,21947885.000000,0.017834,159805,2540,620.886719,635788.000000,0.000000,231.695312,237256.000000,634288.000000,0.000000,178608.000000,nan,nan,[],2023-04-01T18:37:49,"['susbrock', 'model_DB', 't760hyy4']",None,0.877315,931.362000,nan,148.500000,231.695312,174.421875,0.931362
1,mk7xepzt,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1NBNHS,finished,21.972 M,https://wandb.ai/susbrock/model_DB/runs/mk7xepzt,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1NBNHS,44.987 M,0.7,2,96,47075466.000000,3,21971645.000000,l5ll32pl64b0.25.MV1NBNHS,0.024055,165117,47075466.000000,943.859375,304128.000000,21971645.000000,0.024055,162533,2584,653.968750,669664.000000,0.000000,266.382812,272776.000000,1254088.000000,0.000000,342436.000000,nan,nan,[],2023-04-01T18:34:49,"['susbrock', 'model_DB', 'mk7xepzt']",None,0.880792,4463.730000,nan,297.000000,266.382812,334.410156,4.463730
2,db4m6vuv,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBNHS,finished,21.948 M,https://wandb.ai/susbrock/model_DB/runs/db4m6vuv,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBNHS,44.939 M,0.7,2,96,47025570.000000,3,21947885.000000,l5ll32pl32b0.25.MV1NBNHS,0.024049,162345,47025570.000000,934.429688,304128.000000,21947885.000000,0.024049,159805,2540,643.656250,659104.000000,0.000000,262.781250,269088.000000,1253832.000000,0.000000,342176.000000,nan,nan,[],2023-04-01T18:31:22,"['susbrock', 'model_DB', 'db4m6vuv']",None,0.877406,4375.340000,nan,297.000000,262.781250,334.156250,4.375340
3,wtx3h2wq,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBNNA,finished,21.948 M,https://wandb.ai/susbrock/model_DB/runs/wtx3h2wq,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBNNA,45.076 M,0.7,2,96,45075603.000000,3,21947885.000000,l5ll32pl32b0.25.MV1NBNNA,0.017834,162345,45075603.000000,842.632812,152064.000000,21947885.000000,0.017834,159805,2540,620.476562,635368.000000,0.000000,231.234375,236784.000000,634288.000000,0.000000,178608.000000,nan,nan,[],2023-03-31T17:28:58,"['susbrock', 'model_DB', 'wtx3h2wq']",None,0.877315,990.447000,nan,148.500000,231.234375,174.421875,0.990447
4,m8ff44xa,mobilenettiny_0.7_96_c3_o2_l5ll64pl64b0.25.MV1NBNNA,finished,21.980 M,https://wandb.ai/susbrock/model_DB/runs/m8ff44xa,mobilenettiny_0.7_96_c3_o2_l5ll64pl64b0.25.MV1NBNNA,45.142 M,0.7,2,96,45142241.000000,3,21980401.000000,l5ll64pl64b0.25.MV1NBNNA,0.017834,166239,45142241.000000,857.781250,152064.000000,21980401.000000,0.017834,163611,2628,634.738281,649972.000000,0.000000,236.335938,242008.000000,634800.000000,0.000000,179124.000000,nan,nan,[],2023-03-31T17:27:15,"['susbrock', 'model_DB', 'm8ff44xa']",None,0.878865,1001.310000,nan,148.500000,236.335938,174.925781,1.001310
5,5y0sdod4,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1NBNNA,finished,21.972 M,https://wandb.ai/susbrock/model_DB/runs/5y0sdod4,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1NBNNA,45.124 M,0.7,2,96,45123915.000000,3,21971645.000000,l5ll32pl64b0.25.MV1NBNNA,0.017834,165117,45123915.000000,852.062500,152064.000000,21971645.000000,0.017834,162533,2584,630.789062,645928

## Cleanup "tags" column

In [319]:
type(df.tags.value_counts())
df.tags.value_counts()

[]                      214
[basemodel]              13
[too big]                 1
[basemodel, too big]      1
Name: tags, dtype: int64

In [320]:
tags =df["tags"]
tags_df = pd.DataFrame(tags.to_list())
#tags_df.fillna(0, inplace=True)
tags_df.head()

,0,1
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None


In [321]:
df["tag"] = tags_df[0]
df.head()

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag
0,t760hyy4,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBN,finished,21.948 M,https://wandb.ai/susbrock/model_DB/runs/t760hyy4,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBN,45.461 M,0.7,2,96,45460785.0,3,21947885.0,l5ll32pl32b0.25.MV1NBN,0.017834,162345,45460785.0,833.953125,152064.0,21947885.0,0.017834,159805,2540,620.886719,635788.0,0.0,231.695312,237256.0,634288.0,0.0,178608.0,NaN,NaN,[],2023-04-01T18:37:49,"[susbrock, model_DB, t760hyy4]",None,0.877315,931.362,NaN,148.5,231.695312,174.421875,0.931362,None
1,mk7xepzt,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1...,finished,21.972 M,https://wandb.ai/susbrock/model_DB/runs/mk7xepzt,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1...,44.987 M,0.7,2,96,47075466.0,3,21971645.0,l5ll32pl64b0.25.MV1NBNHS,0.024055,165117,47075466.0,943.859375,304128.0,21971645.0,0.024055,162533,2584,653.968750,669664.0,0.0,266.382812,272776.0,1254088.0,0.0,342436.0,NaN,NaN,[],2023-04-01T18:34:49,"[susbrock, model_DB, mk7xepzt]",None,0.880792,4463.730,NaN,297.0,266.382812,334.410156,4.463730,None
2,db4m6vuv,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1...,finished,21.948 M,https://wandb.ai/susbrock/model_DB/runs/db4m6vuv,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1...,44.939 M,0.7,2,96,47025570.0,3,21947885.0,l5ll32pl32b0.25.MV1NBNHS,0.024049,162345,47025570.0,934.429688,304128.0,21947885.0,0.024049,159805,2540,643.656250,659104.0,0.0,262.781250,269088.0,1253832.0,0.0,342176.0,NaN,NaN,[],2023-04-01T18:31:22,"[susbrock, model_DB, db4m6vuv]",None,0.877406,4375.340,NaN,297.0,262.781250,334.156250,4.375340,None
3,wtx3h2wq,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1...,finished,21.948 M,https://wandb.ai/susbrock/model_DB/runs/wtx3h2wq,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1...,45.076 M,0.7,2,96,45075603.0,3,21947885.0,l5ll32pl32b0.25.MV1NBNNA,0.017834,162345,45075603.0,842.632812,152064.0,21947885.0,0.017834,159805,2540,620.476562,635368.0,0.0,231.234375,236784.0,634288.0,0.0,178608.0,NaN,NaN,[],2023-03-31T17:28:58,"[susbrock, model_DB, wtx3h2wq]",None,0.877315,990.447,NaN,148.5,231.234375,174.421875,0.990447,None
4,m8ff44xa,mobilenettiny_0.7_96_c3_o2_l5ll64pl64b0.25.MV1...,finished,21.980 M,https://wandb.ai/susbrock/model_DB/runs/m8ff44xa,mobilenettiny_0.7_96_c3_o2_l5ll64pl64b0.25.MV1...,45.142 M,0.7,2,96,45142241.0,3,21980401.0,l5ll64pl64b0.25.MV1NBNNA,0.017834,166239,45142241.0,857.781250,152064.0,21980401.0,0.017834,163611,2628,634.738281,649972.0,0.0,236.335938,242008.0,634800.0,0.0,179124.0,NaN,NaN,[],2023-03-31T17:27:15,"[susbrock, model_DB, m8ff44xa]",None,0.878865,1001.310,NaN,148.5,236.335938,174.925781,1.001310,None


In [322]:
max_macs = 60000000
max_flash = 256000 /1024
max_ram = 256000 
df["fit"] = True

#df.loc[ (df.tflite_model_INT8_size_kb <=max_flash) | (df.macs_mltk<=max_macs) | (df.RAM_runtime_memory_size_b_INT8_mltk <=max_ram), "fit"] = True
df.loc[df.tflite_model_INT8_size_kb >=max_flash, "fit"] = False
df.loc[df.macs_mltk >=max_macs, "fit"] = False
df.loc[df.RAM_runtime_memory_size_b_INT8_mltk >=max_ram, "fit"] = False

In [323]:
df.head(5)

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag,fit
0,t760hyy4,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBN,finished,21.948 M,https://wandb.ai/susbrock/model_DB/runs/t760hyy4,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBN,45.461 M,0.7,2,96,45460785.0,3,21947885.0,l5ll32pl32b0.25.MV1NBN,0.017834,162345,45460785.0,833.953125,152064.0,21947885.0,0.017834,159805,2540,620.886719,635788.0,0.0,231.695312,237256.0,634288.0,0.0,178608.0,NaN,NaN,[],2023-04-01T18:37:49,"[susbrock, model_DB, t760hyy4]",None,0.877315,931.362,NaN,148.5,231.695312,174.421875,0.931362,None,True
1,mk7xepzt,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1...,finished,21.972 M,https://wandb.ai/susbrock/model_DB/runs/mk7xepzt,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1...,44.987 M,0.7,2,96,47075466.0,3,21971645.0,l5ll32pl64b0.25.MV1NBNHS,0.024055,165117,47075466.0,943.859375,304128.0,21971645.0,0.024055,162533,2584,653.968750,669664.0,0.0,266.382812,272776.0,1254088.0,0.0,342436.0,NaN,NaN,[],2023-04-01T18:34:49,"[susbrock, model_DB, mk7xepzt]",None,0.880792,4463.730,NaN,297.0,266.382812,334.410156,4.463730,None,False
2,db4m6vuv,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1...,finished,21.948 M,https://wandb.ai/susbrock/model_DB/runs/db4m6vuv,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1...,44.939 M,0.7,2,96,47025570.0,3,21947885.0,l5ll32pl32b0.25.MV1NBNHS,0.024049,162345,47025570.0,934.429688,304128.0,21947885.0,0.024049,159805,2540,643.656250,659104.0,0.0,262.781250,269088.0,1253832.0,0.0,342176.0,NaN,NaN,[],2023-04-01T18:31:22,"[susbrock, model_DB, db4m6vuv]",None,0.877406,4375.340,NaN,297.0,262.781250,334.156250,4.375340,None,False
3,wtx3h2wq,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1...,finished,21.948 M,https://wandb.ai/susbrock/model_DB/runs/wtx3h2wq,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1...,45.076 M,0.7,2,96,45075603.0,3,21947885.0,l5ll32pl32b0.25.MV1NBNNA,0.017834,162345,45075603.0,842.632812,152064.0,21947885.0,0.017834,159805,2540,620.476562,635368.0,0.0,231.234375,236784.0,634288.0,0.0,178608.0,NaN,NaN,[],2023-03-31T17:28:58,"[susbrock, model_DB, wtx3h2wq]",None,0.877315,990.447,NaN,148.5,231.234375,174.421875,0.990447,None,True
4,m8ff44xa,mobilenettiny_0.7_96_c3_o2_l5ll64pl64b0.25.MV1...,finished,21.980 M,https://wandb.ai/susbrock/model_DB/runs/m8ff44xa,mobilenettiny_0.7_96_c3_o2_l5ll64pl64b0.25.MV1...,45.142 M,0.7,2,96,45142241.0,3,21980401.0,l5ll64pl64b0.25.MV1NBNNA,0.017834,166239,45142241.0,857.781250,152064.0,21980401.0,0.017834,163611,2628,634.738281,649972.0,0.0,236.335938,242008.0,634800.0,0.0,179124.0,NaN,NaN,[],2023-03-31T17:27:15,"[susbrock, model_DB, m8ff44xa]",None,0.878865,1001.310,NaN,148.5,236.335938,174.925781,1.001310,None,True


In [324]:
df.fit.value_counts()

True     147
False     82
Name: fit, dtype: int64

In [325]:
#df.test_accuracy.value_counts

In [326]:
df.columns

Index(['id', 'run_name', 'state', 'MACs', 'url', 'architecture', 'FLOPs',
       'alpha', 'classes', 'img_res', 'OPS_mltk', 'channels', 'macs_mltk',
       'variation', 'energy_mltk', 'total_params', 'OPS_INT8_mltk',
       'model_size_kb', 'peak_memory_b', 'macs_INT8_mltk', 'energy_INT8_mltk',
       'trainable_params', 'non_trainable_params', 'tflite_model_size_kb',
       'flash_model_size_b_mltk', 'n_unsupported_layers_mltk',
       'tflite_model_INT8_size_kb', 'flash_model_size_b__INT8_mltk',
       'RAM_runtime_memory_size_b_mltk', 'n_unsupported_layers_INT8_mltk',
       'RAM_runtime_memory_size_b_INT8_mltk', 'opt_RAM_mltk',
       'opt_RAM_INT8_mltk', 'tags', 'created_at', 'path', 'notes',
       'test_accuracy', 'inference_avg_us', 'stm32_inference_ms_INT8',
       'peak_memory_kb', 'flash_model_size_kb_INT8_mltk',
       'RAM_runtime_memory_size_kb_INT8_mltk', 'inference_GPU_ms', 'tag',
       'fit'],
      dtype='object')

In [327]:
# check for redundant models in the model_DB
df["architecture"].value_counts()

mobilenetv3small_1_224_c3_o2_000                       2
mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBN      1
mobilenetv2_0.25_96_c3_o2_t6l256.MV1                   1
mobilenetv2_0.2_96_c3_o2_t1l64.MV1                     1
mobilenetv2_0.2_96_c3_o2_t2l64.MV1                     1
mobilenetv2_0.2_96_c3_o2_t3l64.MV1                     1
mobilenetv2_0.2_96_c3_o2_t4l64.MV1                     1
mobilenetv2_0.2_96_c3_o2_t5l64.MV1                     1
mobilenetv2_0.2_96_c3_o2_t6l64.MV1                     1
mobilenetv2_0.2_96_c3_o2_t3l1024.MV1                   1
mobilenetv2_0.2_96_c3_o2_t4l1024.MV1                   1
mobilenetv2_0.25_96_c3_o2_t5l256.MV1                   1
mobilenetv2_0.25_96_c3_o2_t5l128.MV1                   1
mobilenetv2_0.25_96_c3_o2_t6l128.MV1                   1
mobilenetv2_0.25_96_c3_o2_t6l512.MV1                   1
mobilenetv2_0.25_96_c3_o2_t5l64.MV1                    1
mobilenetv2_0.2_96_c3_o2_t6l512.MV1                    1
mobilenetv2_0.2_96_c3_o2_t6l256

# Helper functions

In [328]:
# def get_model_DB_run_id_from_architecture(architecture):
#     df  = wandb_model_DB()

#     run_id = df.query(f"architecture=='{architecture}'")["id"].values[0]
#     return run_id

In [329]:
id = get_model_DB_run_id_from_architecture("efficientNetB0_0.05_96_c3_o2_keras")
id

'vtrfqgnv'

In [330]:
# get_architecture_from_model_DB_run_iddef (run_id):
#     # get model name from run_id
#     df  = wandb_model_DB()
#     model_name = df.query(f"id=='{run_id}'")['architecture'].values[0]
#     return model_name

In [331]:
myname = get_architecture_from_model_DB_run_id(id)
myname

'efficientNetB0_0.05_96_c3_o2_keras'

In [332]:
# training_csv_path = Path.cwd().joinpath("model_DB_visual_wake_words.csv")
# training_csv_path.is_file()
# # df = pd.read_csv(training_csv_path)
# # df

# Plotting functions

In [333]:
template = "plotly_white"#  ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]
bar_width = 0.25
height = 700

In [334]:
def get_model_family_df(model_family):

    
    model_list = df[["architecture"]].query(f"architecture.str.contains('{model_family}', na=False)").copy()
    model_list = model_list["architecture"].tolist()
    #print(model_list)
    family_df = df.query("architecture in @model_list").copy()
    family_df.sort_values(by="alpha", ascending=True, inplace=True)
    family_df["MACs_kB"]=create_kb_column(family_df, "MACs")
    print(f"Number of models: {len(family_df)}")
    
    return family_df

In [335]:
def memory_fig(df_long):

    peak_memory_limit = 87
    flash_memory_limit = 250
    fig = px.bar(df_long, x="architecture", color="size",
                y='value',
                labels={"architecture": "model architecture",
                            #"size": "size",
                            #"value": "kB"
                            },
                title="Peak memory, RAM size & flash size",
                barmode='group',
                #hover_data=['architecture', 'size'],
                height=600,
                template=template
                )
    fig.add_hline(y=peak_memory_limit, line_width=2, line_dash="dash", line_color="orange", annotation_text=f"{peak_memory_limit} KB peak activation", 
                annotation_position="top left")
    fig.add_hline(y=flash_memory_limit, line_width=2, line_dash="dash", line_color="blue", annotation_text=f"{flash_memory_limit} KB flash memory", 
                annotation_position="top right")
    fig.show()
    return fig

In [336]:
def create_size_df_long(df):
    size_df = df[["architecture", 
             "peak_memory_kb", 
             "RAM_runtime_memory_size_kb_INT8_mltk",
             "flash_model_size_kb_INT8_mltk",
             "tflite_model_INT8_size_kb"
             ]].copy()
    df_long = pd.melt(size_df, id_vars=["architecture"], var_name= "size", value_name="value").sort_values(by=["architecture","size"])
    return df_long

In [337]:
def plot_accuracy_macs_by_model_family(model_family):
    family_df = get_model_family_df(model_family)
    macs_accuracy_fig = px.scatter(
    family_df,
    x = "macs_mltk",
    #x="MACs",
    #x="MACs_kb",
    y="test_accuracy",
    color="alpha",
    symbol="fit",
    symbol_sequence=["circle", "cross"],
    # get marker = "fit"
    hover_data=["run_name","alpha", "macs_mltk", "total_params", "peak_memory_kb"],
    #size="tflite_model_INT8_size_kb",
    #labels={"total_params": "parameters", "OPS_INT8_mltk": "FLOPs"},
    title=f"Accuracy vs. MACs - {model_family}",
    template=template
    )
    macs_accuracy_fig.update_layout(xaxis_range=[0, 15000000],
                                    yaxis_range= [0.75, 0.9],
                                    )
    return macs_accuracy_fig


# Get base models

In [425]:
basemodels = df.query(f"tag.str.contains('basemodel', na=False)").copy()
basemodels.sort_values("architecture", axis=0, inplace=True, key=lambda col: col.str.lower())
# max_accuracy = basemodels.test_accuracy.max()
# basemodels["leader"] = basemodels.test_accuracy = max_accuracy
basemodels

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag,fit
97,syyajz7h,efficientNetLiteB0_1_96_c3_o2_000,finished,70.516 M,https://wandb.ai/susbrock/model_DB/runs/syyajz7h,efficientNetLiteB0_1_96_c3_o2_000,143.333 M,1.0,2,96,147322620.0,3,70516192.0,000,0.057820,3415586,147322620.0,13799.734375,451680.0,70516192.0,0.057820,3373570,42016,13063.960938,13377496.0,0.0,3804.296875,3895600.0,1985256.0,0.0,630808.0,NaN,NaN,[basemodel],2023-03-12T21:40:01,"[susbrock, model_DB, syyajz7h]",,0.000000,4013.140,NaN,441.093750,3804.296875,616.023438,4.013140,basemodel,False
45,yr1iql6l,effnet_0.3_96_c3_o2_000,finished,50.467 M,https://wandb.ai/susbrock/model_DB/runs/yr1iql6l,effnet_0.3_96_c3_o2_000,101.450 M,0.3,2,96,106094602.0,3,50466816.0,000,0.053131,186146,106094602.0,861.601562,589824.0,50466816.0,0.053131,183906,2240,726.199219,743628.0,0.0,223.492188,228856.0,2374944.0,0.0,605960.0,NaN,NaN,[basemodel],2023-03-23T19:52:48,"[susbrock, model_DB, yr1iql6l]",,0.851409,10033.100,643.613,576.000000,223.492188,591.757812,10.033100,basemodel,False
202,zhmiturf,mnasnet_1_96_c3_o2_keras,finished,49.649 M,https://wandb.ai/susbrock/model_DB/runs/zhmiturf,mnasnet_1_96_c3_o2_keras,104.940 M,1.0,2,96,215129021.0,3,103265824.0,keras,0.086874,4269716,215277629.0,18749.273438,204352.0,103265824.0,0.087141,4232978,36738,16580.343750,16978272.0,0.0,5150.984375,5274608.0,1195500.0,0.0,569764.0,NaN,NaN,[basemodel],2023-02-19T08:39:21,"[susbrock, model_DB, zhmiturf]",,0.000000,19160.600,NaN,199.562500,5150.984375,556.410156,19.160600,basemodel,False
224,i4nnlhl8,mobilenetv1_0.25_96_c3_o2_l5,finished,7.490 M,https://wandb.ai/susbrock/model_DB/runs/i4nnlhl8,mobilenetv1_0.25_96_c3_o2_l5,15.674 M,0.25,2,96,15673996.0,3,7489664.0,l5,0.005974,221794,15784596.0,1123.593750,55296.0,7489664.0,0.006173,216322,5472,834.507812,854536.0,0.0,294.171875,301232.0,322644.0,0.0,101956.0,NaN,NaN,[basemodel],2023-02-05T20:59:40,"[susbrock, model_DB, i4nnlhl8]",,0.849859,1549.330,NaN,54.000000,294.171875,99.566406,1.549330,basemodel,False
139,l679i489,mobilenetv1vvw_0.25_96_c3_o2_vvw,finished,7.490 M,https://wandb.ai/susbrock/model_DB/runs/l679i489,mobilenetv1vvw_0.25_96_c3_o2_vvw,15.677 M,0.25,2,96,15676554.0,3,7489664.0,vvw,0.005933,221794,15787154.0,1121.812500,55296.0,7489664.0,0.006132,216322,5472,834.285156,854308.0,0.0,294.046875,301104.0,322200.0,0.0,101512.0,NaN,NaN,[basemodel],2023-03-05T17:09:22,"[susbrock, model_DB, l679i489]",,0.851696,502.384,66.360,54.000000,294.046875,99.132812,0.502384,basemodel,False
206,r7tokmwg,mobilenetv2_0.1_96_c3_o2_l5,finished,1.692 M,https://wandb.ai/susbrock/model_DB/runs/r7tokmwg,mobilenetv2_0.1_96_c3_o2_l5,3.643 M,0.1,2,96,3778886.0,3,1692166.0,l5,0.003322,77494,3889486.0,760.898438,55296.0,1692166.0,0.003521,71900,5594,295.679688,302776.0,0.0,176.492188,180728.0,218192.0,0.0,108936.0,NaN,NaN,[basemodel],2023-02-11T21:12:26,"[susbrock, model_DB, r7tokmwg]",,0.000000,1015.370,NaN,54.000000,176.492188,106.382812,1.015370,basemodel,True
207,8i2yq4ji,mobilenetv2_0.2_96_c3_o2_l5,finished,3.636 M,https://wandb.ai/susbrock/model_DB/runs/8i2yq4ji,mobilenetv2_0.2_96_c3_o2_l5,7.729 M,0.2,2,96,7973498.0,3,3635914.0,l5,0.004632,181586,8084098.0,1167.500000,55296.0,3635914.0,0.004831,172886,8700,679.386719,695692.0,0.0,309.632812,317064.0,253368.0,0.0,121132.0,NaN,NaN,[basemodel],2023-02-11T21:09:23,"[susbrock, model_DB, 8i2yq4

In [426]:
def plot_model_parameters(df, title=""):
    fig = px.bar(df, 
                x="architecture", 
                color= "architecture",  # "leader"
                y='total_params',
                labels={"architecture": "model architecture",
                            #"size": "size",
                            #"value": "kB"
                            },
                title=f"total parameters {title}",
                barmode='group',
                #hover_data=['architecture', 'size'],
                height=height,
                template=template
                )

    fig.update_traces(width=bar_width)
    #fig.show()
    return fig

plot_model_parameters(basemodels, title=" - base models")

In [427]:
def plot_model_size(df, title=""):
    peak_memory_limit = 256
    flash_memory_limit = 256
    fig = px.bar(df, 
                x="architecture", 
                color="architecture",
                y='tflite_model_INT8_size_kb',
                labels={"architecture": "model architecture",
                        "tflite_model_INT8_size_kb": "kB",
                            #"value": "kB"
                            },
                title=f"model size in kB {title}",
                barmode='group',
                #hover_data=['architecture', 'size'],
                height=600,
                template=template
                )
    #fig.add_hline(y=peak_memory_limit, line_width=2, line_dash="dash", line_color="orange", annotation_text=f"{peak_memory_limit} KB peak activation", 
    #            annotation_position="top left")
    fig.add_hline(y=flash_memory_limit, line_width=2, line_dash="dash", line_color="blue", annotation_text=f"{flash_memory_limit} KB flash memory", 
                annotation_position="top right")
    fig.update_traces(width=bar_width)
    #fig.show()
    return fig

plot_model_size(basemodels, title=" - base models")

In [428]:
def plot_model_MACs(df, title=""):
    MACs_limit = 60000000
    fig = px.bar(df, 
                x="architecture", 
                color="architecture",
                y='macs_mltk',
                labels={"architecture": "model architecture",
                        "macs_mltk": "MACs",
                            #"value": "kB"
                            },
                title=f"MACs {title}",
                barmode='group',
                #hover_data=['architecture', 'size'],
                height=height,
                template=template
                )
    #fig.add_hline(y=peak_memory_limit, line_width=2, line_dash="dash", line_color="orange", annotation_text=f"{peak_memory_limit} KB peak activation", 
    #            annotation_position="top left")
    fig.add_hline(y=MACs_limit, line_width=2, line_dash="dash", line_color="blue", annotation_text=f"{MACs_limit} MACs", 
                annotation_position="top right")
    fig.update_traces(width=bar_width)
    #fig.show()
    return fig

plot_model_MACs(basemodels, title=" - base models")

In [429]:


def plot_model_RAM(df, title=""):
    peak_memory_limit = 256
    flash_memory_limit = 256
    RAM_limit = 256
    fig = px.bar(df, 
                x="architecture", 
                color="architecture",
                y='RAM_runtime_memory_size_kb_INT8_mltk',
                labels={"architecture": "model architecture",
                        "RAM_runtime_memory_size_kb_INT8_mltk": "kB",
                            #"value": "kB"
                            },
                title=f"RAM in kB {title}",
                barmode='group',
                #hover_data=['architecture', 'size'],
                height=height,
                template=template
                )
    #fig.add_hline(y=peak_memory_limit, line_width=2, line_dash="dash", line_color="orange", annotation_text=f"{peak_memory_limit} KB peak activation", 
    #            annotation_position="top left")
    fig.add_hline(y=RAM_limit, line_width=2, line_dash="dash", line_color="blue", annotation_text=f"{RAM_limit} kB RAM", 
                annotation_position="top right")
    fig.update_traces(width=bar_width)
    #fig.show()
    return fig

plot_model_RAM(basemodels, title=" - base models")

In [430]:


def plot_model_peak_memory(df, title=""):
    peak_memory_limit = 256
    flash_memory_limit = 256
    RAM_limit = 256
    fig = px.bar(df, 
                x="architecture", 
                color="architecture",
                y='peak_memory_kb',
                labels={"architecture": "model architecture",
                        "peak_memory_kb": "kB",
                            #"value": "kB"
                            },
                title=f"peak memory in kB {title}",
                barmode='group',
                #hover_data=['architecture', 'size'],
                height=height,
                template=template
                )
    #fig.add_hline(y=peak_memory_limit, line_width=2, line_dash="dash", line_color="orange", annotation_text=f"{peak_memory_limit} KB peak activation", 
    #            annotation_position="top left")
    fig.add_hline(y=peak_memory_limit, line_width=2, line_dash="dash", line_color="blue", annotation_text=f"{peak_memory_limit} peak memory kB", 
                annotation_position="top right")
    fig.update_traces(width=bar_width)
    #fig.show()
    return fig

plot_model_peak_memory(basemodels, title=" - base models")

In [431]:
def plot_model_accuracy(df, title=""):
    accuracy_limit = 0.80

    fig = px.bar(df, 
                x="architecture", 
                color="architecture",
                y='test_accuracy',
                labels={"architecture": "model architecture",
                        "test_accuracy": "test accuracy %",
                            #"value": "kB"
                            },
                title=f"test accuracy {title}",
                barmode='group',
                #hover_data=['architecture', 'size'],
                height=height,
                template=template
                )
    #fig.add_hline(y=peak_memory_limit, line_width=2, line_dash="dash", line_color="orange", annotation_text=f"{peak_memory_limit} KB peak activation", 
    #            annotation_position="top left")
    fig.add_hline(y=accuracy_limit, line_width=2, line_dash="dash", line_color="blue", annotation_text=f"{accuracy_limit} accuracy %", 
                annotation_position="top right")
    fig.update_traces(width=bar_width)
    fig.update_layout(yaxis_range= [0.75, 0.9])
    #fig.show()
    return fig

plot_model_accuracy(basemodels, title=" - base models")

In [432]:
def plot_model_latency_cpu(df, title=""):
    accuracy_limit = 0.80

    fig = px.bar(df, 
                x="architecture", 
                color="architecture",
                y='inference_GPU_ms',
                labels={"architecture": "model architecture",
                        "inference_GPU_ms": "ms",
                            #"value": "kB"
                            },
                title=f"average inference time on GPU in ms {title}",
                barmode='group',
                #hover_data=['architecture', 'size'],
                height=height,
                template=template
                )
    #fig.add_hline(y=peak_memory_limit, line_width=2, line_dash="dash", line_color="orange", annotation_text=f"{peak_memory_limit} KB peak activation", 
    #            annotation_position="top left")
    # fig.add_hline(y=accuracy_limit, line_width=2, line_dash="dash", line_color="blue", annotation_text=f"{accuracy_limit} accuracy %", 
    #             annotation_position="top right")
    fig.update_traces(width=bar_width)
    # fig.update_layout(yaxis_range= [0.75, 0.9])
    #fig.show()
    return fig

plot_model_latency_cpu(basemodels, title=" - base models")

In [433]:
def plot_model_latency_mcu(df, title=""):
    accuracy_limit = 0.80

    fig = px.bar(df, 
                x="architecture", 
                color="architecture",
                y='stm32_inference_ms_INT8',
                labels={"architecture": "model architecture",
                        "stm32_inference_ms_INT8": "ms",
                            #"value": "kB"
                            },
                title=f"average inference time on MCU in ms {title}",
                barmode='group',
                #hover_data=['architecture', 'size'],
                height=height,
                template=template
                )
    #fig.add_hline(y=peak_memory_limit, line_width=2, line_dash="dash", line_color="orange", annotation_text=f"{peak_memory_limit} KB peak activation", 
    #            annotation_position="top left")
    # fig.add_hline(y=accuracy_limit, line_width=2, line_dash="dash", line_color="blue", annotation_text=f"{accuracy_limit} accuracy %", 
    #             annotation_position="top right")
    fig.update_traces(width=bar_width)
    # fig.update_layout(yaxis_range= [0.75, 0.9])
    #fig.show()
    return fig

plot_model_latency_mcu(basemodels, title=" - base models")

In [434]:
# basemodel_fig = go.Figure(data=[go.Bar(
#                                         name=
# )])

# Leaderboard Analysis

In [435]:
df_top = df[df["fit"]==True].copy()
#df_top

In [436]:
leaderboard = df_top.nlargest(20,'test_accuracy').copy()
leaderboard.sort_values("architecture", axis=0, inplace=True, key=lambda col: col.str.lower())
# max_accuracy = leaderboard.test_accuracy.max()
# leaderboard["leader"] = leaderboard.test_accuracy = max_accuracy
leaderboard

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag,fit
0,t760hyy4,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBN,finished,21.948 M,https://wandb.ai/susbrock/model_DB/runs/t760hyy4,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1NBN,45.461 M,0.7,2,96,45460785.0,3,21947885.0,l5ll32pl32b0.25.MV1NBN,0.017834,162345,45460785.0,833.953125,152064.0,21947885.0,0.017834,159805,2540,620.886719,635788.0,0.0,231.695312,237256.0,634288.0,0.0,178608.0,NaN,NaN,[],2023-04-01T18:37:49,"[susbrock, model_DB, t760hyy4]",None,0.877315,931.362,NaN,148.500000,231.695312,174.421875,0.931362,None,True
3,wtx3h2wq,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1...,finished,21.948 M,https://wandb.ai/susbrock/model_DB/runs/wtx3h2wq,mobilenettiny_0.7_96_c3_o2_l5ll32pl32b0.25.MV1...,45.076 M,0.7,2,96,45075603.0,3,21947885.0,l5ll32pl32b0.25.MV1NBNNA,0.017834,162345,45075603.0,842.632812,152064.0,21947885.0,0.017834,159805,2540,620.476562,635368.0,0.0,231.234375,236784.0,634288.0,0.0,178608.0,NaN,NaN,[],2023-03-31T17:28:58,"[susbrock, model_DB, wtx3h2wq]",None,0.877315,990.447,NaN,148.500000,231.234375,174.421875,0.990447,None,True
7,0454dorp,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1,finished,21.972 M,https://wandb.ai/susbrock/model_DB/runs/0454dorp,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1,45.509 M,0.7,2,96,45509493.0,3,21971645.0,l5ll32pl64b0.25.MV1,0.017834,170197,45509493.0,923.363281,152064.0,21971645.0,0.017834,165073,5124,635.292969,650540.0,0.0,239.406250,245152.0,634544.0,0.0,178868.0,NaN,NaN,[],2023-03-31T13:06:14,"[susbrock, model_DB, 0454dorp]",None,0.882970,987.464,253.119,148.500000,239.406250,174.675781,0.987464,None,True
6,uo5am42b,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1NBN,finished,21.972 M,https://wandb.ai/susbrock/model_DB/runs/uo5am42b,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1NBN,45.509 M,0.7,2,96,45509493.0,3,21971645.0,l5ll32pl64b0.25.MV1NBN,0.017834,165117,45509493.0,845.382812,152064.0,21971645.0,0.017834,162533,2584,631.199219,646348.0,0.0,235.304688,240952.0,634544.0,0.0,178868.0,NaN,NaN,[],2023-03-31T17:22:38,"[susbrock, model_DB, uo5am42b]",None,0.880234,968.921,253.123,148.500000,235.304688,174.675781,0.968921,None,True
5,5y0sdod4,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1...,finished,21.972 M,https://wandb.ai/susbrock/model_DB/runs/5y0sdod4,mobilenettiny_0.7_96_c3_o2_l5ll32pl64b0.25.MV1...,45.124 M,0.7,2,96,45123915.0,3,21971645.0,l5ll32pl64b0.25.MV1NBNNA,0.017834,165117,45123915.0,852.062500,152064.0,21971645.0,0.017834,162533,2584,630.789062,645928.0,0.0,234.843750,240480.0,634544.0,0.0,178868.0,NaN,NaN,[],2023-03-31T17:25:26,"[susbrock, model_DB, 5y0sdod4]",None,0.879504,935.923,252.277,148.500000,234.843750,174.675781,0.935923,None,True
8,mqe2vl35,mobilenettiny_0.7_96_c3_o2_l5ll64pl64b0.25.MV1,finished,21.980 M,https://wandb.ai/susbrock/model_DB/runs/mqe2vl35,mobilenettiny_0.7_96_c3_o2_l5ll64pl64b0.25.MV1,45.528 M,0.7,2,96,45527819.0,3,21980401.0,l5ll64pl64b0.25.MV1,0.017834,171319,45527819.0,927.031250,152064.0,21980401.0,0.017834,166151,5168,639.246094,654588.0,0.0,240.906250,246688.0,634800.0,0.0,179124.0,NaN,NaN,[],2023-03-30T18:44:26,"[susbrock, model_DB, mqe2vl35]",None,0.872845,998.069,253.206,148.500000,240.906250,174.925781,0.998069,None,True
9,s8kdsifp,mobilenettiny_0.7_96_c3_o2_l5ll64pl64b0.25.MV1NBN,finished,21.980 M,https://wandb.ai/susbrock/model_DB/runs/s8kdsifp,m

In [437]:
plot_model_parameters(leaderboard, title=" - leaderboard")


In [438]:
plot_model_size(leaderboard, title=" - leaderboard")

In [439]:
plot_model_MACs(leaderboard, title=" - leaderboard")

In [440]:
plot_model_RAM(leaderboard, title=" - leaderboard")

In [441]:
plot_model_peak_memory(leaderboard, title=" - leaderboard")

In [442]:
plot_model_accuracy(leaderboard, title=" - leaderboard")

In [356]:
plot_model_latency_cpu(leaderboard, title=" - leaderboard")

In [357]:
#leaderboard.architecture.to_list()

In [358]:
plot_model_latency_mcu(leaderboard, title=" - leaderboard")

# MobileNetv V1 Analysis

In [359]:
# mobilenetv1_list = df[["architecture"]].query("architecture.str.contains('mobilenetv1', na=False)").copy()
# mobilenetv1_list = mobilenetv1_list["architecture"].tolist()
# mobilenetv1_list

In [360]:
mbnv1_df = get_model_family_df("mobilenetv1")
mbnv1_df.head()

Number of models: 76


,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag,fit,MACs_kB
185,lum2673e,mobilenetv1_0.1_224_c3_o2_l5.GAP,finished,7.877 M,https://wandb.ai/susbrock/model_DB/runs/lum2673e,mobilenetv1_0.1_224_c3_o2_l5.GAP,17.203 M,0.1,2,224,17203193.0,3,7877395.0,l5.GAP,0.017652,41171,17805313.0,418.878906,301056.0,7877395.0,0.017902,39003,2168,159.527344,163356.0,0.0,85.500000,87552.0,767088.0,0.0,316080.0,NaN,NaN,[],2023-02-22T20:34:10,"[susbrock, model_DB, lum2673e]",None,0.000000,1729.070,NaN,294.0,85.500000,308.671875,1.729070,None,False,8066.048
117,61xlkd8r,mobilenetv1_0.1_96_c3_o2_l4.MV1,finished,1.337 M,https://wandb.ai/susbrock/model_DB/runs/61xlkd8r,mobilenetv1_0.1_96_c3_o2_l4.MV1,2.930 M,0.1,2,96,2929795.0,3,1336875.0,l4.MV1,0.001596,37601,3040395.0,385.351562,55296.0,1336875.0,0.001795,35637,1964,145.992188,149496.0,0.0,78.093750,79968.0,151432.0,0.0,69100.0,NaN,NaN,[],2023-03-09T06:02:30,"[susbrock, model_DB, 61xlkd8r]",None,0.800237,479.403,NaN,54.0,78.093750,67.480469,0.479403,None,True,1369.088
118,iqin4gdq,mobilenetv1_0.1_96_c3_o2_l3.MV1,finished,1.227 M,https://wandb.ai/susbrock/model_DB/runs/iqin4gdq,mobilenetv1_0.1_96_c3_o2_l3.MV1,2.698 M,0.1,2,96,2698459.0,3,1226715.0,l3.MV1,0.001536,34031,2809059.0,353.011719,55296.0,1226715.0,0.001735,32271,1760,132.777344,135964.0,0.0,71.054688,72760.0,150264.0,0.0,67836.0,NaN,NaN,[],2023-03-09T06:00:31,"[susbrock, model_DB, iqin4gdq]",None,0.803795,414.650,NaN,54.0,71.054688,66.246094,0.414650,None,True,1256.448
205,sd3qtst4,mobilenetv1_0.1_96_c3_o2_l5.GAP,finished,1.447 M,https://wandb.ai/susbrock/model_DB/runs/sd3qtst4,mobilenetv1_0.1_96_c3_o2_l5.GAP,3.160 M,0.1,2,96,3160113.0,3,1447035.0,l5.GAP,0.001695,41171,3270713.0,418.722656,55296.0,1447035.0,0.001895,39003,2168,159.320312,163144.0,0.0,85.281250,87328.0,152688.0,0.0,70320.0,NaN,NaN,[],2023-02-18T10:08:46,"[susbrock, model_DB, sd3qtst4]",None,0.801879,392.904,NaN,54.0,85.281250,68.671875,0.392904,None,True,1481.728
187,t3imun41,mobilenetv1_0.1_96_c3_o2_l5.MV1,finished,1.447 M,https://wandb.ai/susbrock/model_DB/runs/t3imun41,mobilenetv1_0.1_96_c3_o2_l5.MV1,3.161 M,0.1,2,96,3161131.0,3,1447035.0,l5.MV1,0.001656,41171,3271731.0,416.699219,55296.0,1447035.0,0.001855,39003,2168,159.207031,163028.0,0.0,85.132812,87176.0,152600.0,0.0,70236.0,NaN,NaN,[],2023-02-22T19:53:41,"[susbrock, model_DB, t3imun41]",None,0.799872,330.124,NaN,54.0,85.132812,68.589844,0.330124,None,True,1481.728


In [361]:
mbnv1_df.columns

Index(['id', 'run_name', 'state', 'MACs', 'url', 'architecture', 'FLOPs',
       'alpha', 'classes', 'img_res', 'OPS_mltk', 'channels', 'macs_mltk',
       'variation', 'energy_mltk', 'total_params', 'OPS_INT8_mltk',
       'model_size_kb', 'peak_memory_b', 'macs_INT8_mltk', 'energy_INT8_mltk',
       'trainable_params', 'non_trainable_params', 'tflite_model_size_kb',
       'flash_model_size_b_mltk', 'n_unsupported_layers_mltk',
       'tflite_model_INT8_size_kb', 'flash_model_size_b__INT8_mltk',
       'RAM_runtime_memory_size_b_mltk', 'n_unsupported_layers_INT8_mltk',
       'RAM_runtime_memory_size_b_INT8_mltk', 'opt_RAM_mltk',
       'opt_RAM_INT8_mltk', 'tags', 'created_at', 'path', 'notes',
       'test_accuracy', 'inference_avg_us', 'stm32_inference_ms_INT8',
       'peak_memory_kb', 'flash_model_size_kb_INT8_mltk',
       'RAM_runtime_memory_size_kb_INT8_mltk', 'inference_GPU_ms', 'tag',
       'fit', 'MACs_kB'],
      dtype='object')

In [362]:
mbnv1_df["MACs_kB"]=create_kb_column(mbnv1_df, "MACs")
mbnv1_df.head()

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag,fit,MACs_kB
185,lum2673e,mobilenetv1_0.1_224_c3_o2_l5.GAP,finished,7.877 M,https://wandb.ai/susbrock/model_DB/runs/lum2673e,mobilenetv1_0.1_224_c3_o2_l5.GAP,17.203 M,0.1,2,224,17203193.0,3,7877395.0,l5.GAP,0.017652,41171,17805313.0,418.878906,301056.0,7877395.0,0.017902,39003,2168,159.527344,163356.0,0.0,85.500000,87552.0,767088.0,0.0,316080.0,NaN,NaN,[],2023-02-22T20:34:10,"[susbrock, model_DB, lum2673e]",None,0.000000,1729.070,NaN,294.0,85.500000,308.671875,1.729070,None,False,8066.048
117,61xlkd8r,mobilenetv1_0.1_96_c3_o2_l4.MV1,finished,1.337 M,https://wandb.ai/susbrock/model_DB/runs/61xlkd8r,mobilenetv1_0.1_96_c3_o2_l4.MV1,2.930 M,0.1,2,96,2929795.0,3,1336875.0,l4.MV1,0.001596,37601,3040395.0,385.351562,55296.0,1336875.0,0.001795,35637,1964,145.992188,149496.0,0.0,78.093750,79968.0,151432.0,0.0,69100.0,NaN,NaN,[],2023-03-09T06:02:30,"[susbrock, model_DB, 61xlkd8r]",None,0.800237,479.403,NaN,54.0,78.093750,67.480469,0.479403,None,True,1369.088
118,iqin4gdq,mobilenetv1_0.1_96_c3_o2_l3.MV1,finished,1.227 M,https://wandb.ai/susbrock/model_DB/runs/iqin4gdq,mobilenetv1_0.1_96_c3_o2_l3.MV1,2.698 M,0.1,2,96,2698459.0,3,1226715.0,l3.MV1,0.001536,34031,2809059.0,353.011719,55296.0,1226715.0,0.001735,32271,1760,132.777344,135964.0,0.0,71.054688,72760.0,150264.0,0.0,67836.0,NaN,NaN,[],2023-03-09T06:00:31,"[susbrock, model_DB, iqin4gdq]",None,0.803795,414.650,NaN,54.0,71.054688,66.246094,0.414650,None,True,1256.448
205,sd3qtst4,mobilenetv1_0.1_96_c3_o2_l5.GAP,finished,1.447 M,https://wandb.ai/susbrock/model_DB/runs/sd3qtst4,mobilenetv1_0.1_96_c3_o2_l5.GAP,3.160 M,0.1,2,96,3160113.0,3,1447035.0,l5.GAP,0.001695,41171,3270713.0,418.722656,55296.0,1447035.0,0.001895,39003,2168,159.320312,163144.0,0.0,85.281250,87328.0,152688.0,0.0,70320.0,NaN,NaN,[],2023-02-18T10:08:46,"[susbrock, model_DB, sd3qtst4]",None,0.801879,392.904,NaN,54.0,85.281250,68.671875,0.392904,None,True,1481.728
187,t3imun41,mobilenetv1_0.1_96_c3_o2_l5.MV1,finished,1.447 M,https://wandb.ai/susbrock/model_DB/runs/t3imun41,mobilenetv1_0.1_96_c3_o2_l5.MV1,3.161 M,0.1,2,96,3161131.0,3,1447035.0,l5.MV1,0.001656,41171,3271731.0,416.699219,55296.0,1447035.0,0.001855,39003,2168,159.207031,163028.0,0.0,85.132812,87176.0,152600.0,0.0,70236.0,NaN,NaN,[],2023-02-22T19:53:41,"[susbrock, model_DB, t3imun41]",None,0.799872,330.124,NaN,54.0,85.132812,68.589844,0.330124,None,True,1481.728


In [363]:
mbnv1_96_df = mbnv1_df[["architecture"]].query("architecture.str.contains('_96_', na=False) & architecture.str.contains('_o2', na=False)").copy()
mbnv1_96_df.sort_values("architecture", axis=0, inplace=True)
mbnv1_96_df

,architecture
119,mobilenetv1_0.1_96_c3_o2_l1.MV1
126,mobilenetv1_0.1_96_c3_o2_l2.GAP
162,mobilenetv1_0.1_96_c3_o2_l2.MV1
118,mobilenetv1_0.1_96_c3_o2_l3.MV1
117,mobilenetv1_0.1_96_c3_o2_l4.MV1
205,mobilenetv1_0.1_96_c3_o2_l5.GAP
187,mobilenetv1_0.1_96_c3_o2_l5.MV1
228,mobilenetv1_0.25_96_c3_o2_l1
227,mobilenetv1_0.25_96_c3_o2_l2
226,mobilenetv1_0.25_96_c3_o2_l3


In [364]:
mbnv1_size_df_long = create_size_df_long(mbnv1_df)
#mbnv1_size_df_long

In [365]:
mbnv1_size_df_long = create_size_df_long(mbnv1_df)
mbnv1_memory_fig = memory_fig(mbnv1_size_df_long)

In [366]:
plot_accuracy_macs_by_model_family("mobilenetv1")

Number of models: 76


# MobileNet V2 Analysis

In [367]:
mbnv2_df = get_model_family_df("mobilenetv2")
mbnv2_df.head()

Number of models: 63


,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag,fit,MACs_kB
98,5tonipyt,mobilenetv2_0.1_96_c3_o2_t6l64.MV1,finished,1.340 M,https://wandb.ai/susbrock/model_DB/runs/5tonipyt,mobilenetv2_0.1_96_c3_o2_t6l64.MV1,2.916 M,0.1,2,96,3041412.0,3,1339526.0,t6l64.MV1,0.003230,31286,3041412.0,578.773438,46080.0,1339526.0,0.003230,28124,3162,128.539062,131624.0,0.0,101.328125,103760.0,208008.0,0.0,70564.0,NaN,NaN,[],2023-03-12T21:30:08,"[susbrock, model_DB, 5tonipyt]",None,0.000000,383.829,NaN,45.0,101.328125,68.910156,0.383829,None,True,1372.160
168,bifxqx73,mobilenetv2_0.1_96_c3_o2_t6l512.MV1,finished,1.469 M,https://wandb.ai/susbrock/model_DB/runs/bifxqx73,mobilenetv2_0.1_96_c3_o2_t6l512.MV1,3.189 M,0.1,2,96,3317828.0,3,1469446.0,t6l512.MV1,0.003274,48310,3428428.0,644.851562,55296.0,1469446.0,0.003473,44252,4058,189.789062,194344.0,0.0,128.921875,132016.0,211592.0,0.0,83404.0,NaN,NaN,[],2023-02-25T10:52:42,"[susbrock, model_DB, bifxqx73]",None,0.000000,372.385,NaN,54.0,128.921875,81.449219,0.372385,None,True,1504.256
170,eitbmrqm,mobilenetv2_0.1_96_c3_o2_t6l128.MV1,finished,1.358 M,https://wandb.ai/susbrock/model_DB/runs/eitbmrqm,mobilenetv2_0.1_96_c3_o2_t6l128.MV1,2.955 M,0.1,2,96,3080900.0,3,1358086.0,t6l128.MV1,0.003230,33718,3191500.0,588.773438,55296.0,1358086.0,0.003429,30428,3290,137.289062,140584.0,0.0,105.671875,108208.0,208520.0,0.0,80332.0,NaN,NaN,[],2023-02-25T10:47:42,"[susbrock, model_DB, eitbmrqm]",None,0.000000,360.126,NaN,54.0,105.671875,78.449219,0.360126,None,True,1390.592
171,d4e7lbhl,mobilenetv2_0.1_96_c3_o2_t1l1024.MV1,finished,1.085 M,https://wandb.ai/susbrock/model_DB/runs/d4e7lbhl,mobilenetv2_0.1_96_c3_o2_t1l1024.MV1,2.303 M,0.1,2,96,2370987.0,3,1085441.0,t1l1024.MV1,0.002816,44506,2481587.0,629.351562,55296.0,1085441.0,0.003015,41684,2822,185.238281,189684.0,0.0,114.570312,117320.0,206788.0,0.0,78772.0,NaN,NaN,[],2023-02-25T10:45:12,"[susbrock, model_DB, d4e7lbhl]",None,0.748791,453.978,NaN,54.0,114.570312,76.925781,0.453978,None,True,1111.040
172,yqbs8c2d,mobilenetv2_0.1_96_c3_o2_t2l1024.MV1,finished,1.192 M,https://wandb.ai/susbrock/model_DB/runs/yqbs8c2d,mobilenetv2_0.1_96_c3_o2_t2l1024.MV1,2.543 M,0.1,2,96,2623536.0,3,1191938.0,t2l1024.MV1,0.002853,49158,2734136.0,649.351562,55296.0,1191938.0,0.003052,45884,3274,200.511719,205324.0,0.0,123.593750,126560.0,208544.0,0.0,80332.0,NaN,NaN,[],2023-02-25T10:42:52,"[susbrock, model_DB, yqbs8c2d]",None,0.763021,493.610,NaN,54.0,123.593750,78.449219,0.493610,None,True,1220.608


In [368]:
mbnv2_df.MACs.unique()

array(['1.340 M', '1.469 M', '1.358 M', '1.085 M', '1.192 M', '1.298 M',
       '1.395 M', '1.511 M', '1.618 M', '1.692 M', '1.405 M', '2.762 M',
       '3.636 M', '3.488 M', '3.125 M', '2.829 M', '2.681 M', '2.607 M',
       '3.044 M', '3.192 M', '2.399 M', '2.970 M', '2.570 M', '2.466 M',
       '2.318 M', '2.933 M', '2.103 M', '1.955 M', '1.881 M', '1.119 M',
       '2.244 M', '1.844 M', '1.482 M', '2.207 M', '2.069 M', '5.219 M',
       '3.294 M', '3.478 M', '5.034 M', '4.441 M', '3.848 M', '3.255 M',
       '2.662 M', '4.071 M', '6.600 M', '3.794 M', '4.341 M', '3.886 M',
       '3.155 M', '2.562 M', '3.748 M', '1.376 M', '4.664 M', '4.479 M',
       '4.387 M', '1.969 M', '9.009 M', '10.657 M', '16.125 M',
       '17.608 M', '55.012 M'], dtype=object)

In [369]:
mbnv2_size_df_long = create_size_df_long(mbnv2_df)
mobilenetv2_mem_fig = memory_fig(mbnv2_size_df_long)

In [370]:
for i in range(1,7):

    mbnv2_df.loc[mbnv2_df['run_name'].str.contains(f"t{i}", case=False), "t"] = i
mbnv2_df

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag,fit,MACs_kB,t
98,5tonipyt,mobilenetv2_0.1_96_c3_o2_t6l64.MV1,finished,1.340 M,https://wandb.ai/susbrock/model_DB/runs/5tonipyt,mobilenetv2_0.1_96_c3_o2_t6l64.MV1,2.916 M,0.1,2,96,3041412.0,3,1339526.0,t6l64.MV1,0.003230,31286,3041412.0,578.773438,46080.0,1339526.0,0.003230,28124,3162,128.539062,131624.0,0.0,101.328125,103760.0,208008.0,0.0,70564.0,NaN,NaN,[],2023-03-12T21:30:08,"[susbrock, model_DB, 5tonipyt]",None,0.000000,383.829,NaN,45.000000,101.328125,68.910156,0.383829,None,True,1372.160,6.0
168,bifxqx73,mobilenetv2_0.1_96_c3_o2_t6l512.MV1,finished,1.469 M,https://wandb.ai/susbrock/model_DB/runs/bifxqx73,mobilenetv2_0.1_96_c3_o2_t6l512.MV1,3.189 M,0.1,2,96,3317828.0,3,1469446.0,t6l512.MV1,0.003274,48310,3428428.0,644.851562,55296.0,1469446.0,0.003473,44252,4058,189.789062,194344.0,0.0,128.921875,132016.0,211592.0,0.0,83404.0,NaN,NaN,[],2023-02-25T10:52:42,"[susbrock, model_DB, bifxqx73]",None,0.000000,372.385,NaN,54.000000,128.921875,81.449219,0.372385,None,True,1504.256,6.0
170,eitbmrqm,mobilenetv2_0.1_96_c3_o2_t6l128.MV1,finished,1.358 M,https://wandb.ai/susbrock/model_DB/runs/eitbmrqm,mobilenetv2_0.1_96_c3_o2_t6l128.MV1,2.955 M,0.1,2,96,3080900.0,3,1358086.0,t6l128.MV1,0.003230,33718,3191500.0,588.773438,55296.0,1358086.0,0.003429,30428,3290,137.289062,140584.0,0.0,105.671875,108208.0,208520.0,0.0,80332.0,NaN,NaN,[],2023-02-25T10:47:42,"[susbrock, model_DB, eitbmrqm]",None,0.000000,360.126,NaN,54.000000,105.671875,78.449219,0.360126,None,True,1390.592,6.0
171,d4e7lbhl,mobilenetv2_0.1_96_c3_o2_t1l1024.MV1,finished,1.085 M,https://wandb.ai/susbrock/model_DB/runs/d4e7lbhl,mobilenetv2_0.1_96_c3_o2_t1l1024.MV1,2.303 M,0.1,2,96,2370987.0,3,1085441.0,t1l1024.MV1,0.002816,44506,2481587.0,629.351562,55296.0,1085441.0,0.003015,41684,2822,185.238281,189684.0,0.0,114.570312,117320.0,206788.0,0.0,78772.0,NaN,NaN,[],2023-02-25T10:45:12,"[susbrock, model_DB, d4e7lbhl]",None,0.748791,453.978,NaN,54.000000,114.570312,76.925781,0.453978,None,True,1111.040,1.0
172,yqbs8c2d,mobilenetv2_0.1_96_c3_o2_t2l1024.MV1,finished,1.192 M,https://wandb.ai/susbrock/model_DB/runs/yqbs8c2d,mobilenetv2_0.1_96_c3_o2_t2l1024.MV1,2.543 M,0.1,2,96,2623536.0,3,1191938.0,t2l1024.MV1,0.002853,49158,2734136.0,649.351562,55296.0,1191938.0,0.003052,45884,3274,200.511719,205324.0,0.0,123.593750,126560.0,208544.0,0.0,80332.0,NaN,NaN,[],2023-02-25T10:42:52,"[susbrock, model_DB, yqbs8c2d]",None,0.763021,493.610,NaN,54.000000,123.593750,78.449219,0.493610,None,True,1220.608,2.0
173,f8jsvkj6,mobilenetv2_0.1_96_c3_o2_t3l1024.MV1,finished,1.298 M,https://wandb.ai/susbrock/model_DB/runs/f8jsvkj6,mobilenetv2_0.1_96_c3_o2_t3l1024.MV1,2.782 M,0.1,2,96,2876085.0,3,1298435.0,t3l1024.MV1,0.002935,53810,2986685.0,666.367188,55296.0,1298435.0,0.003134,50084,3726,215.406250,220576.0,0.0,132.890625,136080.0,210464.0,0.0,82356.0,NaN,NaN,[],2023-02-25T10:40:40,"[susbrock, model_DB, f8jsvkj6]",None,0.000000,504.294,NaN,54.000000,132.890625,80.425781,0.504294,None,True,1329.152,3.0
169,22bzuzhv,mobilenetv2_0.1_96_c3_o2_t6l256.MV1,finished,1.395 M,https://wandb.ai/susbrock/model_DB/runs/22bzuzhv,mobilenetv2_0.1_96_c3_o2_t6l256.MV1,3.033 M,0.1,2,96,3159876.0,3,1395206.0,t6l256.MV1,0.003230,38582,3270476.0,606.851562,55296.0,1395206.0,0.003429,35036,3546,154.789062,158504.0,0.0,113.421875,116144.0,209544.0,0.0,81356.

In [371]:
mbnv2_df_t4 = mbnv2_df.loc[mbnv2_df['t']==4]
mbnv2_df_t4

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag,fit,MACs_kB,t
174,6oq96cks,mobilenetv2_0.1_96_c3_o2_t4l1024.MV1,finished,1.405 M,https://wandb.ai/susbrock/model_DB/runs/6oq96cks,mobilenetv2_0.1_96_c3_o2_t4l1024.MV1,3.021 M,0.1,2,96,3128634.0,3,1404932.0,t4l1024.MV1,0.003029,58462,3239234.0,684.351562,55296.0,1404932.0,0.003228,54284,4178,230.234375,235760.0,0.0,141.710938,145112.0,211848.0,0.0,83688.0,NaN,NaN,[],2023-02-25T10:38:38,"[susbrock, model_DB, 6oq96cks]",None,0.000000,490.063,NaN,54.0,141.710938,81.726562,0.490063,None,True,1438.720,4.0
153,a2wlivv7,mobilenetv2_0.2_96_c3_o2_t4l1024.MV1,finished,2.762 M,https://wandb.ai/susbrock/model_DB/runs/a2wlivv7,mobilenetv2_0.2_96_c3_o2_t4l1024.MV1,5.870 M,0.2,2,96,6052770.0,3,2762204.0,t4l1024.MV1,0.004026,133522,6163370.0,977.968750,55296.0,2762204.0,0.004225,127174,6348,507.332031,519508.0,0.0,237.046875,242736.0,220424.0,0.0,92160.0,NaN,NaN,[],2023-03-01T19:27:13,"[susbrock, model_DB, a2wlivv7]",None,0.815470,614.349,NaN,54.0,237.046875,90.000000,0.614349,None,True,2828.288,4.0
133,hj37ihh4,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,finished,2.466 M,https://wandb.ai/susbrock/model_DB/runs/hj37ihh4,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,5.263 M,0.2,2,96,5441954.0,3,2466268.0,t4l512.MV1,0.004035,97682,5552554.0,837.968750,55296.0,2466268.0,0.004235,92358,5324,373.332031,382292.0,0.0,190.046875,194608.0,216328.0,0.0,88064.0,NaN,NaN,[],2023-03-05T17:21:43,"[susbrock, model_DB, hj37ihh4]",None,0.810545,422.506,NaN,54.0,190.046875,86.000000,0.422506,None,True,2525.184,4.0
134,q8pfj6j8,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,finished,2.318 M,https://wandb.ai/susbrock/model_DB/runs/q8pfj6j8,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,4.960 M,0.2,2,96,5136546.0,3,2318300.0,t4l256.MV1,0.003990,79762,5247146.0,767.968750,55296.0,2318300.0,0.004189,74950,4812,306.332031,313684.0,0.0,166.546875,170544.0,214280.0,0.0,86016.0,NaN,NaN,[],2023-03-05T17:19:40,"[susbrock, model_DB, q8pfj6j8]",None,0.805345,529.504,NaN,54.0,166.546875,84.000000,0.529504,None,True,2373.632,4.0
135,j8oowdud,mobilenetv2_0.2_96_c3_o2_t4l128.MV1,finished,2.244 M,https://wandb.ai/susbrock/model_DB/runs/j8oowdud,mobilenetv2_0.2_96_c3_o2_t4l128.MV1,4.809 M,0.2,2,96,4983842.0,3,2244316.0,t4l128.MV1,0.003962,70802,5094442.0,733.890625,55296.0,2244316.0,0.004161,66246,4556,272.269531,278804.0,0.0,154.351562,158056.0,213268.0,0.0,84992.0,NaN,NaN,[],2023-03-05T17:17:36,"[susbrock, model_DB, j8oowdud]",None,0.815379,405.606,NaN,54.0,154.351562,83.000000,0.405606,None,True,2297.856,4.0
149,wau3gd5m,mobilenetv2_0.2_96_c3_o2_t4l64.MV1,finished,2.207 M,https://wandb.ai/susbrock/model_DB/runs/wau3gd5m,mobilenetv2_0.2_96_c3_o2_t4l64.MV1,4.733 M,0.2,2,96,4907490.0,3,2207324.0,t4l64.MV1,0.003962,66322,5018090.0,715.882812,55296.0,2207324.0,0.004161,61894,4428,255.734375,261872.0,0.0,148.695312,152264.0,212756.0,0.0,84480.0,NaN,NaN,[],2023-03-02T21:16:48,"[susbrock, model_DB, wau3gd5m]",None,0.807352,533.374,NaN,54.0,148.695312,82.500000,0.533374,None,True,2259.968,4.0
129,j48pxbt0,mobilenetv2_0.25_96_c3_o2_t4l256.MV1,finished,3.294 M,https://wandb.ai/susbrock/model_DB/runs/j48pxbt0,mobilenetv2_0.25_96_c3_o2_t4l256.MV1,6.999 M,0.25,2,96,7224546.0,3,3293504.0,t4l256.MV1,0.004609,115474,7335146.0,908.445312,55296.0,3293504.0,0.004808,109442,6032,436.628906,447108.0,0.0,213.890625,219024.0,219168.0,0.0,90900.0,NaN,NaN,[],2023-03-05T1

In [372]:
### TODO: Clean up accuracy plots
macs_accuracy_fig = px.scatter(
    mbnv2_df,
    x = "macs_mltk",
    #x="MACs",
    y="test_accuracy",
    color="alpha",
    hover_data=["run_name","alpha", "macs_mltk", "total_params", "peak_memory_kb"],
    #size="tflite_model_INT8_size_kb",
    #labels={"total_params": "parameters", "OPS_INT8_mltk": "FLOPs"},
    title=f"Accuracy vs. MACs",
)
macs_accuracy_fig.update_layout(showlegend=True, width=1200, height=400)

macs_accuracy_fig.show()

In [373]:

macs_accuracy_fig = px.scatter(
mbnv2_df_t4,
x = "macs_mltk",
#x="MACs",
#x="MACs_kb",
y="test_accuracy",
color="alpha",
hover_data=["run_name","alpha", "macs_mltk", "total_params", "peak_memory_kb"],
#size="tflite_model_INT8_size_kb",
#labels={"total_params": "parameters", "OPS_INT8_mltk": "FLOPs"},
title=f"Accuracy vs. MACs" # - {model_family}",
)

macs_accuracy_fig.show()

In [374]:

# macs_params_fig = px.scatter(
#     mbnv2_df,
#     x="macs_mltk",
#     y="tflite_model_INT8_size_kb",
#     color="alpha",
#     size="total_params",
#     hover_data=["run_name","alpha", "macs_mltk", "total_params"],
#     labels={"macs_mltk": "MACs", 
#             "tflite_model_INT8_size_kb" : "INT8 model size in kB",
#             "total_params": "parameters"},
#     title=f"INT8 model size vs. MACs",
# )

# macs_params_fig.update_layout(showlegend=True, width=500, height=400)

# macs_params_fig.show()

In [375]:
def params_macs_alpha_fig(df, model_name):
    df["alpha"] = df["alpha"].astype(str)
    fig = px.scatter(
        df,
        x="total_params",
        y="macs_mltk",
        color="alpha",
        hover_data=["run_name","alpha", "macs_mltk", "total_params"],
        size="tflite_model_INT8_size_kb",
        labels={"total_params": "parameters", 
                "macs_mltk": "MACs",
                "run_name": "model name"},
        title=f"MACs vs. total parameters - {model_name}",
    )
    fig.add_hline(y=60000000, line_width=2, line_dash="dash", line_color="red", annotation_text=f"60M MACs", 
               annotation_position="top")
    fig.update_layout(showlegend=True, width=1200, height=400)
    
    fig.show()
    return fig


In [376]:
mbnv2_fig = params_macs_alpha_fig(mbnv2_df, "MobileNet V2")

In [377]:
flops_params_fig = px.scatter(
    mbnv2_df,
    x="total_params",
    y="OPS_INT8_mltk",
    color="alpha",
    size="tflite_model_INT8_size_kb",
    hover_data=["run_name","alpha", "macs_mltk", "total_params"],
    labels={"total_params": "parameters", "OPS_INT8_mltk": "FLOPs"},
    title=f"FLOPs vs. total parameters",
)
flops_params_fig.update_layout(showlegend=True, width=500, height=400)

flops_params_fig.show()

In [378]:
# df["peak_memory_kb"] = df["peak_memory_b"]/1024.0
# df["flash_model_size_kb_INT8_mltk"] = df["flash_model_size_b__INT8_mltk"]/1024.0
# df["RAM_runtime_memory_size_kb_INT8_mltk"] = df["RAM_runtime_memory_size_b_INT8_mltk"]/1024.0

# Plotting construction zone

In [379]:
# size_fig = px.bar(mbnv2_df, x="alpha", y= ["peak_memory_kb",
#                                            "RAM_runtime_memory_size_kb_INT8_mltk",
#                                            "flash_model_size_kb_INT8_mltk",
#                                            "tflite_model_INT8_size_kb"], 
#                   text_auto=".2s",
#                   labels={"name" : "layer",
#                           "macs" : "MACs"},
#                      width=1200, height=600,
#     title=f"Model size")
# size_fig.add_hline(y=256, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#             annotation_position="bottom right")
# #peak_memory = peak_memory_df["RAM_b"].max()
# #peak_mem_fig.add_hline(y=peak_memory, line_width=3, line_dash="dash", line_color="red", annotation_text=f"{peak_memory} B peak activation", 
# #               annotation_position="top")
# # tflite_fig.update_layout(yaxis_range=[0,300])
# size_fig.update_traces(width=1)
# size_fig.update_layout(showlegend=True)

# size_fig.show()

In [380]:
# # Create figure with secondary y-axis
# fig = make_subplots(specs=[[{"secondary_y": True}]])
# #fig = go.Figure()#specs=[[{"secondary_y": True}]])
# # Add traces
# fig.add_trace(go.Bar(x = mbnv2_df["alpha"],
#                                    y = mbnv2_df["tflite_model_INT8_size_kb"],#, mbnv2_df["peak_memory_b"]], 
#                                    name= "model size in kB"
#                                    ),
#                                    col=1,
#                                    row=1,
#                                    secondary_y=False)

# fig.add_trace(go.Bar(x = mbnv2_df["alpha"],
#                                    y = mbnv2_df["peak_memory_b"], 
#                                    name= "peak memory in B"
#                                    ),
#                                    col=1,
#                                    row=1,
#                                    secondary_y=True)



# # fig.add_trace(
# #     go.Scatter(x=[2, 3, 4], y=[4, 5, 6], name="yaxis2 data"),
# #     secondary_y=True,
# # )

# # Add figure title
# fig.update_layout(
#     title_text="Double Y Axis Example"
# )

# # Set x-axis title
# fig.update_xaxes(title_text="xaxis title")

# # Set y-axes titles
# fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
# fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)
# fig.update_layout(barmode="group",
#                   bargap=0.15, # gap between bars of adjacent location coordinates.
#                     bargroupgap=0.1 # gap between bars of the same location coordinate.)
# )
# fig.show()

wandb.log({"my_chart": wandb.Html(plotly.io.to_html(fig)})

# ShuffleNet V2 tiny Analysis

In [381]:
shufflenetv2tiny_list = df[["architecture"]].query("architecture.str.contains('shufflenetv2tiny')").copy()
shufflenetv2tiny_list = shufflenetv2tiny_list["architecture"].tolist()
shufflenetv2tiny_list

['shufflenetv2tiny_0.2_96_c3_o2_f16l256',
 'shufflenetv2tiny_0.2_96_c3_o2_f16l512',
 'shufflenetv2tiny_0.2_96_c3_o2_f16l1024',
 'shufflenetv2tiny_0.1_96_c3_o2_f8l64',
 'shufflenetv2tiny_0.1_96_c3_o2_f8l128',
 'shufflenetv2tiny_0.1_96_c3_o2_f8l256',
 'shufflenetv2tiny_0.1_96_c3_o2_f8l512',
 'shufflenetv2tiny_0.1_96_c3_o2_f8l1024',
 'shufflenetv2tiny_0.1_96_c3_o2_f16l64',
 'shufflenetv2tiny_0.1_96_c3_o2_f16l128',
 'shufflenetv2tiny_0.1_96_c3_o2_f16l256',
 'shufflenetv2tiny_0.1_96_c3_o2_f16l512',
 'shufflenetv2tiny_0.1_96_c3_o2_f16l1024',
 'shufflenetv2tiny_0.1_96_c3_o2_f24l64',
 'shufflenetv2tiny_0.1_96_c3_o2_f24l256',
 'shufflenetv2tiny_0.1_96_c3_o2_f24l512',
 'shufflenetv2tiny_0.05_96_c3_o2_f24l128',
 'shufflenetv2tiny_0.1_96_c3_o2_f24l128',
 'shufflenetv2tiny_0.25_96_c3_o2_f24l128',
 'shufflenetv2tiny_0.25_96_c3_o2_f24l512',
 'shufflenetv2tiny_0.05_96_c3_o2_f24l1024',
 'shufflenetv2tiny_0.1_96_c3_o2_f24l1024',
 'shufflenetv2tiny_0.2_96_c3_o2_f24l1024',
 'shufflenetv2tiny_0.25_96_c3_o2

In [382]:
shufflev2tiny_df = get_model_family_df("shufflenetv2tiny")
shufflev2tiny_df

Number of models: 26


,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag,fit,MACs_kB
193,x4xe6px9,shufflenetv2tiny_0.05_96_c3_o2_f24l1024,finished,2.289 M,https://wandb.ai/susbrock/model_DB/runs/x4xe6px9,shufflenetv2tiny_0.05_96_c3_o2_f24l1024,4.851 M,0.05,2,96,4946618.0,3,2288624.0,f24l1024,0.003780,65602,5057218.0,962.578125,80664.0,2288624.0,0.003979,64054,1548,313.828125,321360.0,0.0,187.265625,191760.0,364904.0,0.0,124464.0,NaN,NaN,[],2023-02-19T14:15:09,"[susbrock, model_DB, x4xe6px9]",None,0.000000,1039.41,NaN,78.773438,187.265625,121.546875,1.03941,None,True,2343.936
189,zmlnt7qa,shufflenetv2tiny_0.05_96_c3_o2_f24l128,finished,1.900 M,https://wandb.ai/susbrock/model_DB/runs/zmlnt7qa,shufflenetv2tiny_0.05_96_c3_o2_f24l128,4.073 M,0.05,2,96,4160826.0,3,1899760.0,f24l128,0.003729,20802,4271426.0,786.578125,80664.0,1899760.0,0.003928,19254,1548,135.328125,138576.0,0.0,119.015625,121872.0,357736.0,0.0,117296.0,NaN,NaN,[],2023-02-20T19:14:20,"[susbrock, model_DB, zmlnt7qa]",None,0.771231,1016.12,NaN,78.773438,119.015625,114.546875,1.01612,None,True,1945.600
113,6klpgxte,shufflenetv2tiny_0.1_96_c3_o2_f16l1024,finished,2.960 M,https://wandb.ai/susbrock/model_DB/runs/6klpgxte,shufflenetv2tiny_0.1_96_c3_o2_f16l1024,6.161 M,0.1,2,96,6267722.0,3,2960384.0,f16l1024,0.003003,143322,6378322.0,1269.406250,62992.0,2960384.0,0.003202,140354,2968,606.855469,621420.0,0.0,277.492188,284152.0,299728.0,0.0,112320.0,NaN,NaN,[],2023-03-10T12:10:43,"[susbrock, model_DB, 6klpgxte]",None,0.000000,1886.32,NaN,61.515625,277.492188,109.687500,1.88632,None,False,3031.040
116,zj6udmpw,shufflenetv2tiny_0.1_96_c3_o2_f24l512,finished,3.074 M,https://wandb.ai/susbrock/model_DB/runs/zj6udmpw,shufflenetv2tiny_0.1_96_c3_o2_f24l512,6.464 M,0.1,2,96,6585234.0,3,3073624.0,f24l512,0.003864,93690,6695834.0,1075.937500,80664.0,3073624.0,0.004064,90690,3000,410.656250,420512.0,0.0,215.375000,220544.0,366428.0,0.0,126016.0,NaN,NaN,[],2023-03-10T11:48:34,"[susbrock, model_DB, zj6udmpw]",None,0.000000,1737.17,NaN,78.773438,215.375000,123.062500,1.73717,None,True,3147.776
115,uxejz7y3,shufflenetv2tiny_0.1_96_c3_o2_f24l256,finished,2.852 M,https://wandb.ai/susbrock/model_DB/runs/uxejz7y3,shufflenetv2tiny_0.1_96_c3_o2_f24l256,6.021 M,0.1,2,96,6139538.0,3,2851928.0,f24l256,0.003886,68602,6250138.0,977.937500,80664.0,2851928.0,0.004086,65602,3000,311.656250,319136.0,0.0,183.875000,188288.0,364380.0,0.0,123968.0,NaN,NaN,[],2023-03-10T11:52:30,"[susbrock, model_DB, uxejz7y3]",None,0.000000,1910.16,NaN,78.773438,183.875000,121.062500,1.91016,None,True,2920.448
114,6a4ds1fm,shufflenetv2tiny_0.1_96_c3_o2_f24l64,finished,2.686 M,https://wandb.ai/susbrock/model_DB/runs/6a4ds1fm,shufflenetv2tiny_0.1_96_c3_o2_f24l64,5.688 M,0.1,2,96,5805266.0,3,2685656.0,f24l64,0.003817,49786,5915866.0,903.937500,80664.0,2685656.0,0.004016,46786,3000,237.406250,243104.0,0.0,160.250000,164096.0,362844.0,0.0,122432.0,NaN,NaN,[],2023-03-10T11:56:15,"[susbrock, model_DB, 6a4ds1fm]",None,0.000000,1953.82,NaN,78.773438,160.250000,119.562500,1.95382,None,True,2750.464
112,jl9fkae9,shufflenetv2tiny_0.1_96_c3_o2_f16l512,finished,2.517 M,https://wandb.ai/susbrock/model_DB/runs/jl9fkae9,shufflenetv2tiny_0.1_96_c3_o2_f16l512,5.274 M,0.1,2,96,5376330.0,3,2516992.0,f16l512,0.002927,93146,5486930.0,1073.406250,62992.0,2516992.0,0.003127,90178,2968,408.855469,418668.0,0.0,214.492188,219640.0,295632.0,0.0,108224.0,

In [383]:
shufflev2tiny_df_long = create_size_df_long(shufflev2tiny_df)
shufflev2tiny_mem_fig = memory_fig(shufflev2tiny_df_long)

In [384]:
plot_accuracy_macs_by_model_family("shufflenetv2tiny")

Number of models: 26


In [385]:
### TODO: Clean up accuracy plots
macs_accuracy_fig = px.scatter(
    shufflev2tiny_df,
    x = "macs_mltk",
    y="test_accuracy",
    color="alpha",
    hover_data=["run_name","alpha", "macs_mltk", "total_params", "peak_memory_kb"],
    #size="tflite_model_INT8_size_kb",
    #labels={"total_params": "parameters", "OPS_INT8_mltk": "FLOPs"},
    title=f"Accuracy vs. MACs - ShuffleNetV2Tiny",
)
macs_accuracy_fig.update_layout(showlegend=True, width=1200, height=400)

macs_accuracy_fig.show()

# Analysis of single architectures

In [386]:
architecture = "mobilenetv1_0.1_96_c3_o2_l2.MV1"

In [387]:
run_id =  get_model_DB_run_id_from_architecture(architecture)
run_id

'f6keiwns'

In [388]:
# get model name from run_id
model_name = df.query(f"id=='{run_id}'")['architecture'].values[0]
model_name

'mobilenetv1_0.1_96_c3_o2_l2.MV1'

# Comparison of two models

In [389]:
architecture_1 = "mobilenetv2_0.25_96_c3_o2_l5"
run_id_1 = get_model_DB_run_id_from_architecture(architecture_1)
architecture_2 = "mobilenetv2_0.25_96_c3_o2_keras"
run_id_2 = get_model_DB_run_id_from_architecture(architecture_2)

In [390]:
run_id_2

'upj2bfr1'

In [391]:
ENTITY = "susbrock"
PROJECT ="model_DB"
#run_id = "27bt3quc"
table_name = "tflite_layers_mltk"

In [392]:
df_1 = get_wandb_table_as_df(run_id_1, table_name)
df_1.head()

wandb:   1 of 1 files downloaded.  


filepath .\artifacts\run-to56zsaq-tflite_layers_mltk-v0/tflite_layers_mltk.table.json


,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,conv_2d,0: conv_2d,1050624,497664,4.600492e+06,0.000836,"1x96x96x3,8x3x3x3,8",1x48x48x8,Padding:same stride:2x2 activation:relu6
1,1,depthwise_conv_2d,1: depthwise_conv_2d,387072,165888,2.522040e+06,0.000468,"1x48x48x8,1x3x3x8,8",1x48x48x8,Multiplier:1 padding:same stride:1x1 activatio...
2,2,conv_2d,2: conv_2d,156672,73728,8.374511e+05,0.000113,"1x48x48x8,4x1x1x8,4",1x48x48x4,Padding:same stride:1x1 activation:none
3,3,conv_2d,3: conv_2d,608256,221184,4.586929e+06,0.000833,"1x48x48x4,24x1x1x4,24",1x48x48x24,Padding:valid stride:1x1 activation:relu6
4,4,depthwise_conv_2d,4: depthwise_conv_2d,290304,124416,1.528548e+06,0.000294,"1x48x48x24,1x3x3x24,24",1x24x24x24,Multiplier:1 padding:same stride:2x2 activatio...


In [393]:
df_2 = get_wandb_table_as_df(run_id_2, table_name)
df_2.head()

wandb:   1 of 1 files downloaded.  


filepath .\artifacts\run-upj2bfr1-tflite_layers_mltk-v0/tflite_layers_mltk.table.json


,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,conv_2d,0: conv_2d,1050624,497664,4.600492e+06,0.000836,"1x96x96x3,8x3x3x3,8",1x48x48x8,Padding:same stride:2x2 activation:relu6
1,1,depthwise_conv_2d,1: depthwise_conv_2d,387072,165888,2.522040e+06,0.000468,"1x48x48x8,1x3x3x8,8",1x48x48x8,Multiplier:1 padding:same stride:1x1 activatio...
2,2,conv_2d,2: conv_2d,313344,147456,1.678246e+06,0.000264,"1x48x48x8,8x1x1x8,8",1x48x48x8,Padding:same stride:1x1 activation:none
3,3,add,3: add,18432,0,1.550003e+06,0.000322,"1x48x48x8,1x48x48x8",1x48x48x8,Activation:none
4,4,conv_2d,4: conv_2d,2101248,884736,1.073103e+07,0.002051,"1x48x48x8,48x1x1x8,48",1x48x48x48,Padding:same stride:1x1 activation:relu6


In [394]:
def plot_macs_per_layer(architecture):
    #architecture_1 = "mobilenetv2_0.25_96_c3_o2_l5"
    run_id = get_model_DB_run_id_from_architecture(architecture)
    df = get_wandb_table_as_df(run_id, "tflite_layers_mltk")
    macs_fig = px.bar(df, x="name", y= "macs",
                  text_auto=".2s",
                  labels={"name" : "layer",
                          "macs" : "MACs"},
                     width=1200, height=600,
                     template=template,
    title=f"MACs per layer - tflite {architecture}")
    macs_fig.update_traces(width=1)
    macs_fig.update_layout(showlegend=False)

    #macs_fig.show()
    return macs_fig


In [395]:
def input_height_per_layer(architecture):
    run_id = get_model_DB_run_id_from_architecture(architecture)
    df = get_wandb_table_as_df(run_id, "tensorflow_layer_details")
    macs_fig = px.bar(df, x="layer_name", y= "h_i_1",
                  text_auto=".2s",
                  labels={"name" : "layer",
                          "h_i_1" : "input height"},
                     width=1200, height=600,
                     template=template,
    title=f"Input height per layer - tflite {architecture}")
    macs_fig.update_traces(width=1)
    macs_fig.update_layout(showlegend=False)

    #macs_fig.show()
    return macs_fig

In [396]:
input_height_per_layer(architecture_1)

wandb:   1 of 1 files downloaded.  


filepath .\artifacts\run-to56zsaq-tensorflow_layer_details-v0/tensorflow_layer_details.table.json


In [397]:
plot_macs_per_layer(architecture_1)

wandb:   1 of 1 files downloaded.  


filepath .\artifacts\run-to56zsaq-tflite_layers_mltk-v0/tflite_layers_mltk.table.json


In [398]:
def plot_macs_per_layer_comparison(architecture_1, architecture_2):
    run_id_1 = get_model_DB_run_id_from_architecture(architecture_1)
    run_id_2 = get_model_DB_run_id_from_architecture(architecture_2)
    df_1 = get_wandb_table_as_df(run_id_1, "tflite_layers_mltk")
    df_2 = get_wandb_table_as_df(run_id_2, "tflite_layers_mltk")

    # dict for the dataframes and their names
    dfs = {architecture_1 : df_1, architecture_2 : df_2}

    fig = go.Figure(layout_title_text=f"MACs per layer overlay - {architecture_1} vs {architecture_2}")

    for i in dfs:
        fig = fig.add_trace(go.Bar(x = dfs[i]["index"],
                                    y = dfs[i]["macs"], 
                                    name = i))
    return fig

In [399]:
plot_macs_per_layer_comparison(architecture_1, architecture_2)

wandb:   1 of 1 files downloaded.  


filepath .\artifacts\run-to56zsaq-tflite_layers_mltk-v0/tflite_layers_mltk.table.json


wandb:   1 of 1 files downloaded.  


filepath .\artifacts\run-upj2bfr1-tflite_layers_mltk-v0/tflite_layers_mltk.table.json


# ShuffleNet V1

In [400]:
shufflenetv1_df = get_model_family_df("shufflenetv1")
shufflenetv1_df

Number of models: 7


,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag,fit,MACs_kB
93,svzac8u0,shufflenetv1_0.2_96_c3_o2_g1,finished,2.564 M,https://wandb.ai/susbrock/model_DB/runs/svzac8u0,shufflenetv1_0.2_96_c3_o2_g1,5.655 M,0.2,2,96,5596202.0,3,2564222.0,g1,0.003779,46265,5596202.0,819.265625,82944.0,2564222.0,0.003779,43371,2894,212.781250,217888.0,0.0,143.664062,147112.0,366972.0,0.0,119148.0,NaN,NaN,[],2023-03-13T06:10:57,"[susbrock, model_DB, svzac8u0]",None,0.850588,1267.86,62.867,81.0,143.664062,116.355469,1.26786,None,True,2625.536
94,xthh3283,shufflenetv1_0.2_96_c3_o2_g3,finished,4.412 M,https://wandb.ai/susbrock/model_DB/runs/xthh3283,shufflenetv1_0.2_96_c3_o2_g3,9.495 M,0.2,2,96,9394618.0,3,4411968.0,g3,0.004052,51530,9394618.0,841.679688,82944.0,4411968.0,0.004052,46634,4896,212.242188,217336.0,0.0,166.335938,170328.0,374444.0,0.0,126536.0,NaN,NaN,[],2023-03-13T06:07:57,"[susbrock, model_DB, xthh3283]",None,0.829061,5233.71,NaN,81.0,166.335938,123.570312,5.23371,None,True,4517.888
95,0okkp98r,shufflenetv1_0.2_96_c3_o2_g2,finished,3.557 M,https://wandb.ai/susbrock/model_DB/runs/0okkp98r,shufflenetv1_0.2_96_c3_o2_g2,7.726 M,0.2,2,96,7642802.0,3,3556832.0,g2,0.003822,50210,7642802.0,836.882812,82944.0,3556832.0,0.003822,46130,4080,215.183594,220348.0,0.0,157.351562,161128.0,371336.0,0.0,123436.0,NaN,NaN,[],2023-03-13T06:04:46,"[susbrock, model_DB, 0okkp98r]",None,0.806987,4101.43,NaN,81.0,157.351562,120.542969,4.10143,None,True,3642.368
203,nprqo2x8,shufflenetv1_0.25_96_c3_o2_g1,finished,3.185 M,https://wandb.ai/susbrock/model_DB/runs/nprqo2x8,shufflenetv1_0.25_96_c3_o2_g1,6.953 M,0.25,2,96,6877486.0,3,3184560.0,g1,0.003824,71030,6988086.0,917.039062,82944.0,3184560.0,0.004023,67358,3672,301.367188,308600.0,0.0,175.171875,179376.0,370068.0,0.0,122304.0,NaN,NaN,[],2023-02-19T07:47:47,"[susbrock, model_DB, nprqo2x8]",None,0.850771,NaN,NaN,81.0,175.171875,119.437500,NaN,None,True,3261.440
204,2ghwgc4i,shufflenetv1_0.25_96_c3_o2_g8,finished,12.639 M,https://wandb.ai/susbrock/model_DB/runs/2ghwgc4i,shufflenetv1_0.25_96_c3_o2_g8,26.299 M,0.25,2,96,26098666.0,3,12639360.0,g8,0.007699,85226,26209266.0,975.148438,82944.0,12639360.0,0.007898,75434,9792,297.472656,304612.0,0.0,246.375000,252288.0,394028.0,0.0,146172.0,NaN,NaN,[],2023-02-19T07:44:18,"[susbrock, model_DB, 2ghwgc4i]",None,0.838913,NaN,NaN,81.0,246.375000,142.746094,NaN,None,True,12942.336
91,vv4kextb,shufflenetv1_0.35_96_c3_o2_g3,finished,10.085 M,https://wandb.ai/susbrock/model_DB/runs/vv4kextb,shufflenetv1_0.35_96_c3_o2_g3,21.103 M,0.35,2,96,20927326.0,3,10084848.0,g3,0.006636,134798,20927326.0,1168.984375,82944.0,10084848.0,0.006636,126230,8568,503.355469,515436.0,0.0,283.148438,289944.0,389628.0,0.0,141696.0,NaN,NaN,[basemodel],2023-03-13T06:16:37,"[susbrock, model_DB, vv4kextb]",,0.842105,8149.08,NaN,81.0,283.148438,138.375000,8.14908,basemodel,False,10327.040
92,9lm0zws5,shufflenetv1_0.35_96_c3_o2_g1,finished,4.619 M,https://wandb.ai/susbrock/model_DB/runs/9lm0zws5,shufflenetv1_0.35_96_c3_o2_g1,9.922 M,0.35,2,96,9817285.0,3,4618968.0,g1,0.004159,129884,9817285.0,1147.101562,82944.0,4618968.0,0.004159,124784,5100,518.523438,530968.0,0.0,246.546875,252464.0,375956.0,0.0,128008.0,NaN,NaN,[basemodel],2023-03-13T06:13:55,"[susbrock, model_DB, 9lm0zws5]",,0.838471,2115.63,90.196,81.0,246.546875,125.007812,2.11563,basemodel,True,4729.856


In [401]:
# for shufflenet v1 
def extract_groups_from_name(architecture_name):

    groups = int(architecture_name.split("_")[-1].replace("g", ""))
    return groups

In [402]:
shufflenetv1_df["groups"] = shufflenetv1_df["architecture"].apply(extract_groups_from_name)
shufflenetv1_df

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,opt_RAM_mltk,opt_RAM_INT8_mltk,tags,created_at,path,notes,test_accuracy,inference_avg_us,stm32_inference_ms_INT8,peak_memory_kb,flash_model_size_kb_INT8_mltk,RAM_runtime_memory_size_kb_INT8_mltk,inference_GPU_ms,tag,fit,MACs_kB,groups
93,svzac8u0,shufflenetv1_0.2_96_c3_o2_g1,finished,2.564 M,https://wandb.ai/susbrock/model_DB/runs/svzac8u0,shufflenetv1_0.2_96_c3_o2_g1,5.655 M,0.2,2,96,5596202.0,3,2564222.0,g1,0.003779,46265,5596202.0,819.265625,82944.0,2564222.0,0.003779,43371,2894,212.781250,217888.0,0.0,143.664062,147112.0,366972.0,0.0,119148.0,NaN,NaN,[],2023-03-13T06:10:57,"[susbrock, model_DB, svzac8u0]",None,0.850588,1267.86,62.867,81.0,143.664062,116.355469,1.26786,None,True,2625.536,1
94,xthh3283,shufflenetv1_0.2_96_c3_o2_g3,finished,4.412 M,https://wandb.ai/susbrock/model_DB/runs/xthh3283,shufflenetv1_0.2_96_c3_o2_g3,9.495 M,0.2,2,96,9394618.0,3,4411968.0,g3,0.004052,51530,9394618.0,841.679688,82944.0,4411968.0,0.004052,46634,4896,212.242188,217336.0,0.0,166.335938,170328.0,374444.0,0.0,126536.0,NaN,NaN,[],2023-03-13T06:07:57,"[susbrock, model_DB, xthh3283]",None,0.829061,5233.71,NaN,81.0,166.335938,123.570312,5.23371,None,True,4517.888,3
95,0okkp98r,shufflenetv1_0.2_96_c3_o2_g2,finished,3.557 M,https://wandb.ai/susbrock/model_DB/runs/0okkp98r,shufflenetv1_0.2_96_c3_o2_g2,7.726 M,0.2,2,96,7642802.0,3,3556832.0,g2,0.003822,50210,7642802.0,836.882812,82944.0,3556832.0,0.003822,46130,4080,215.183594,220348.0,0.0,157.351562,161128.0,371336.0,0.0,123436.0,NaN,NaN,[],2023-03-13T06:04:46,"[susbrock, model_DB, 0okkp98r]",None,0.806987,4101.43,NaN,81.0,157.351562,120.542969,4.10143,None,True,3642.368,2
203,nprqo2x8,shufflenetv1_0.25_96_c3_o2_g1,finished,3.185 M,https://wandb.ai/susbrock/model_DB/runs/nprqo2x8,shufflenetv1_0.25_96_c3_o2_g1,6.953 M,0.25,2,96,6877486.0,3,3184560.0,g1,0.003824,71030,6988086.0,917.039062,82944.0,3184560.0,0.004023,67358,3672,301.367188,308600.0,0.0,175.171875,179376.0,370068.0,0.0,122304.0,NaN,NaN,[],2023-02-19T07:47:47,"[susbrock, model_DB, nprqo2x8]",None,0.850771,NaN,NaN,81.0,175.171875,119.437500,NaN,None,True,3261.440,1
204,2ghwgc4i,shufflenetv1_0.25_96_c3_o2_g8,finished,12.639 M,https://wandb.ai/susbrock/model_DB/runs/2ghwgc4i,shufflenetv1_0.25_96_c3_o2_g8,26.299 M,0.25,2,96,26098666.0,3,12639360.0,g8,0.007699,85226,26209266.0,975.148438,82944.0,12639360.0,0.007898,75434,9792,297.472656,304612.0,0.0,246.375000,252288.0,394028.0,0.0,146172.0,NaN,NaN,[],2023-02-19T07:44:18,"[susbrock, model_DB, 2ghwgc4i]",None,0.838913,NaN,NaN,81.0,246.375000,142.746094,NaN,None,True,12942.336,8
91,vv4kextb,shufflenetv1_0.35_96_c3_o2_g3,finished,10.085 M,https://wandb.ai/susbrock/model_DB/runs/vv4kextb,shufflenetv1_0.35_96_c3_o2_g3,21.103 M,0.35,2,96,20927326.0,3,10084848.0,g3,0.006636,134798,20927326.0,1168.984375,82944.0,10084848.0,0.006636,126230,8568,503.355469,515436.0,0.0,283.148438,289944.0,389628.0,0.0,141696.0,NaN,NaN,[basemodel],2023-03-13T06:16:37,"[susbrock, model_DB, vv4kextb]",,0.842105,8149.08,NaN,81.0,283.148438,138.375000,8.14908,basemodel,False,10327.040,3
92,9lm0zws5,shufflenetv1_0.35_96_c3_o2_g1,finished,4.619 M,https://wandb.ai/susbrock/model_DB/runs/9lm0zws5,shufflenetv1_0.35_96_c3_o2_g1,9.922 M,0.35,2,96,9817285.0,3,4618968.0,g1,0.004159,129884,9817285.0,1147.101562,82944.0,4618968.0,0.004159,124784,5100,518.523438,530968.0,0.0,246.546875,252464.0,375956.0,0.0,128008.0,NaN,NaN,[basemodel],2023-03-13T06:13:55,"[susbrock, model_DB, 9lm0zws5]",,0.838471,2115.63,90.196,81.0,246.546875,125.007812,2.11563,basemodel,True,4729.856,1


In [403]:
def params_macs_alpha_groups_fig(df,model_name):
    df["alpha"] = df["alpha"].astype(str)
    df["groups"] = df["groups"].astype(str)
    fig = px.scatter(
        df,
        x="total_params",
        y="macs_mltk",
        color="groups",
        hover_data=["alpha","groups", "macs_mltk", "total_params"],
        symbol= df["alpha"],
        #size="tflite_model_INT8_size_kb",
        labels={"total_params": "parameters", "macs_mltk": "MACs"},
        title=f"MACs vs. total parameters - {model_name}",
    )
    fig.add_hline(y=60000000, line_width=3, line_dash="dash", line_color="red", annotation_text=f"60M MACs", 
               annotation_position="top")
    fig.update_layout(showlegend=True, width=500, height=400)
    
    fig.show()
    return fig

In [404]:
shufflev1_fig2 = params_macs_alpha_groups_fig(shufflenetv1_df, "ShuffleNet V1")

In [405]:
shufflev1_size_df = shufflenetv1_df[["architecture", 
             "peak_memory_kb", 
             "RAM_runtime_memory_size_kb_INT8_mltk",
             "flash_model_size_kb_INT8_mltk",
             "tflite_model_INT8_size_kb"
             ]].copy()

In [406]:
# fig = make_subplots(rows=2, cols=1, start_cell="bottom-left", column_widths=[1400],row_heights=[0.5, 0.5], vertical_spacing=0.3,
#                     subplot_titles=[architecture_1, architecture_2])
# fig = fig.add_trace(go.Bar(x = df_me["name"],
#                                    y = df_me["macs"], 
#                                    name = architecture_1,
                                   
#                                    text=df_me["macs"],
#                                    textposition='auto'),
#                                    row=1, col=1)
# fig = fig.add_trace(go.Bar(x = df_keras["name"],
#                                    y = df_keras["macs"], 
#                                    name = architecture_2),
#                                    row=2, col=1)

# fig.update_layout(
#     height = 1000,
#     width = 1200,
#     title="MACs per layer - tflite",
#     xaxis_title="layer name",
#     yaxis_title="MACs",
#     legend_title="architecture",
#     # font=dict(
#     #     family="Courier New, monospace",
#     #     size=18,
#     #     color="RebeccaPurple"
#     # )
#     legend=dict(
#     x=0,
#     y=1.0
#     )
# )


# fig.update_layout(showlegend=False)
# fig.show()

In [407]:
ENTITY = "susbrock"
PROJECT = "model_DB"
run_id_3 = "fbfsftzr"
table_name ="Runorder"

api = wandb.Api()
artifact = api.artifact(f'{ENTITY}/{PROJECT}/run-{run_id}-{table_name}:latest')
artifact_dir = artifact.download()

wandb:   1 of 1 files downloaded.  


In [408]:
def get_wandb_node_type_summary_df(run_id):
    table_name = "Nodetypesummary"
    api = wandb.Api()
    artifact = api.artifact(f'{ENTITY}/{PROJECT}/run-{run_id}-{table_name}:latest')
    artifact_dir = artifact.download()
    table_path = Path(artifact_dir).joinpath("Node type summary.table.json")
    #print(f"filepath {table_path}")

    with open(table_path) as file:
        json_dict = json.load(file)
        #print(json_dict)
    df = pd.DataFrame(json_dict["data"], columns=json_dict["columns"])

    return df

In [409]:
node_type_summary_df = get_wandb_node_type_summary_df(run_id)
node_type_summary_df

wandb:   1 of 1 files downloaded.  


,Node type,count,avg ms,avg %,cdf %,mem KB,times called
0,QUANTIZE,2,0.142,36.317,36.317,0.0,2
1,"Convolution (NHWC, QC8) DWConv",10,0.108,27.621,63.939,0.0,10
2,"Convolution (NHWC, QC8) GEMM",10,0.072,18.414,82.353,0.0,10
3,"Convolution (NHWC, QC8) IGEMM",1,0.067,17.136,99.488,0.0,1
4,SOFTMAX,1,0.001,0.256,99.744,0.0,1
5,AVERAGE_POOL_2D,1,0.001,0.256,100.000,0.0,1
6,RESHAPE,1,0.000,0.000,100.000,0.0,1
7,"Fully Connected (NC, QS8) GEMM",1,0.000,0.000,100.000,0.0,1


In [410]:
peak_memory_df = get_wandb_table_as_df(run_id_3, "peak_memory")
peak_memory_df.head()

wandb:   1 of 1 files downloaded.  


filepath .\artifacts\run-fbfsftzr-peak_memory-v0/peak_memory.table.json


,name,tensor_IDs,RAM_b,operator,index
0,tfl.quantize,"[0, 131]",55296,tfl.quantize,0
1,conv2d,"[131, 132]",80664,shufflenetv2tiny/re_lu/Relu;shufflenetv2tiny/b...,1
2,max_pooling2d,"[132, 133]",66840,shufflenetv2tiny/max_pooling2d/MaxPool,2
3,conv2d_1,"[133, 134]",41472,shufflenetv2tiny/re_lu_1/Relu;shufflenetv2tiny...,3
4,depthwise_conv2d_1,"[133, 134, 135]",44928,shufflenetv2tiny/batch_normalization_4/FusedBa...,4


In [411]:
RAM_df = peak_memory_df[["index", "name", "RAM_b"]]
RAM_df.head()

,index,name,RAM_b
0,0,tfl.quantize,55296
1,1,conv2d,80664
2,2,max_pooling2d,66840
3,3,conv2d_1,41472
4,4,depthwise_conv2d_1,44928


In [412]:
tensor_df = get_wandb_table_as_df(run_id_3, "tensor_info")
tensor_df.drop(0, axis=0, inplace=True)
tensor_df.reset_index(inplace=True)
tensor_df.head()

wandb:   1 of 1 files downloaded.  


filepath .\artifacts\run-fbfsftzr-tensor_info-v0/tensor_info.table.json


,index,id,name,shape,size_b,name_long
0,1,131,tfl.quantize,"(1, 96, 96, 3)",27648,tfl.quantize
1,2,132,shufflenetv2tiny/conv2d/Conv2D,"(1, 47, 47, 24)",53016,shufflenetv2tiny/re_lu/Relu;shufflenetv2tiny/b...
2,3,133,shufflenetv2tiny/max_pooling2d/MaxPool,"(1, 24, 24, 24)",13824,max_pooling2d
3,4,134,shufflenetv2tiny/conv2d_1/Conv2D,"(1, 24, 24, 48)",27648,shufflenetv2tiny/re_lu_1/Relu;shufflenetv2tiny...
4,5,135,shufflenetv2tiny/depthwise_conv2d_1/depthwise,"(1, 12, 12, 24)",3456,shufflenetv2tiny/batch_normalization_4/FusedBa...


In [413]:
size_df = tensor_df[["index", "name", "size_b"]]
size_df.head()

,index,name,size_b
0,1,tfl.quantize,27648
1,2,shufflenetv2tiny/conv2d/Conv2D,53016
2,3,shufflenetv2tiny/max_pooling2d/MaxPool,13824
3,4,shufflenetv2tiny/conv2d_1/Conv2D,27648
4,5,shufflenetv2tiny/depthwise_conv2d_1/depthwise,3456


In [414]:
INT8_layers_df = get_wandb_table_as_df(run_id_3, "tflite_INt8_layers_mltk")
INT8_layers_df.head()

wandb:   1 of 1 files downloaded.  


filepath .\artifacts\run-fbfsftzr-tflite_INt8_layers_mltk-v0/tflite_INt8_layers_mltk.table.json


,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,quantize,0: quantize,110592,0,9.957829e+05,0.000199,1x96x96x3,1x96x96x3,Type=none
1,1,conv_2d,1: conv_2d,3021912,1431432,1.152572e+07,0.002198,"1x96x96x3,24x3x3x3,24",1x47x47x24,Padding:valid stride:2x2 activation:relu
2,2,max_pool_2d,2: max_pool_2d,124416,0,0.000000e+00,0.000000,1x47x47x24,1x24x24x24,Padding:same stride:2x2 filter:3x3 activation:...
3,3,conv_2d,3: conv_2d,1410048,663552,3.420720e+06,0.000621,"1x24x24x24,48x1x1x24,48",1x24x24x48,Padding:same stride:1x1 activation:relu
4,4,depthwise_conv_2d,4: depthwise_conv_2d,65664,31104,3.602970e+05,0.000069,"1x24x24x24,1x3x3x24,24",1x12x12x24,Multiplier:1 padding:same stride:2x2 activatio...


In [415]:
macs_df = INT8_layers_df[["index", "name", "ops", "macs"]]
macs_df.head()

,index,name,ops,macs
0,0,0: quantize,110592,0
1,1,1: conv_2d,3021912,1431432
2,2,2: max_pool_2d,124416,0
3,3,3: conv_2d,1410048,663552
4,4,4: depthwise_conv_2d,65664,31104


In [416]:
def get_wandb_run_order_df(run_id):
    table_name = "Runorder"
    api = wandb.Api()
    artifact = api.artifact(f'{ENTITY}/{PROJECT}/run-{run_id}-{table_name}:latest')
    artifact_dir = artifact.download()
    table_path = Path(artifact_dir).joinpath("Run order.table.json")
    #print(f"filepath {table_path}")

    with open(table_path) as file:
        json_dict = json.load(file)
        #print(json_dict)
    df = pd.DataFrame(json_dict["data"], columns=json_dict["columns"])

    return df

In [417]:
run_order_df = get_wandb_run_order_df(run_id_3)
run_order_df.reset_index(inplace=True)
run_order_df.head()

wandb:   1 of 1 files downloaded.  


,index,node type,first,avg ms,%,cdf%,mem KB,times called,Name
0,0,QUANTIZE,0.093,0.113,4.730,4.730,0.0,1,[tfl.quantize]:0
1,1,CONV_2D,0.122,0.149,6.257,10.987,0.0,1,[shufflenetv2tiny/re_lu/Relu;shufflenetv2tiny/...
2,2,MAX_POOL_2D,0.049,0.049,2.040,13.027,0.0,1,[shufflenetv2tiny/max_pooling2d/MaxPool]:2
3,3,CONV_2D,0.043,0.050,2.114,15.141,0.0,1,[shufflenetv2tiny/re_lu_1/Relu;shufflenetv2tin...
4,4,DEPTHWISE_CONV_2D,0.026,0.031,1.288,16.430,0.0,1,[shufflenetv2tiny/batch_normalization_4/FusedB...


In [418]:
time_df = run_order_df[["index", "node type", "first", "avg ms", "%"]]
time_df.head()

,index,node type,first,avg ms,%
0,0,QUANTIZE,0.093,0.113,4.730
1,1,CONV_2D,0.122,0.149,6.257
2,2,MAX_POOL_2D,0.049,0.049,2.040
3,3,CONV_2D,0.043,0.050,2.114
4,4,DEPTHWISE_CONV_2D,0.026,0.031,1.288


In [419]:
combined_df = time_df.join(macs_df, on="index", rsuffix="_time")
combined_df = combined_df.join(RAM_df, on="index", rsuffix="_RAM" )
combined_df = combined_df.join(size_df, on="index", rsuffix="_size")
combined_df.head()

,index,node type,first,avg ms,%,index_time,name,ops,macs,index_RAM,name_RAM,RAM_b,index_size,name_size,size_b
0,0,QUANTIZE,0.093,0.113,4.730,0,0: quantize,110592,0,0,tfl.quantize,55296,1,tfl.quantize,27648
1,1,CONV_2D,0.122,0.149,6.257,1,1: conv_2d,3021912,1431432,1,conv2d,80664,2,shufflenetv2tiny/conv2d/Conv2D,53016
2,2,MAX_POOL_2D,0.049,0.049,2.040,2,2: max_pool_2d,124416,0,2,max_pooling2d,66840,3,shufflenetv2tiny/max_pooling2d/MaxPool,13824
3,3,CONV_2D,0.043,0.050,2.114,3,3: conv_2d,1410048,663552,3,conv2d_1,41472,4,shufflenetv2tiny/conv2d_1/Conv2D,27648
4,4,DEPTHWISE_CONV_2D,0.026,0.031,1.288,4,4: depthwise_conv_2d,65664,31104,4,depthwise_conv2d_1,44928,5,shufflenetv2tiny/depthwise_conv2d_1/depthwise,3456


In [420]:
combined_df["time_%"] = (combined_df['avg ms'] / combined_df['avg ms'].sum()) * 100
combined_df["RAM_%"] = (combined_df['RAM_b'] / combined_df['RAM_b'].sum()) * 100
combined_df["size_%"] = (combined_df['size_b'] / combined_df['size_b'].sum()) * 100
combined_df["ops_%"] = (combined_df['ops'] / combined_df['ops'].sum()) * 100
combined_df

,index,node type,first,avg ms,%,index_time,name,ops,macs,index_RAM,name_RAM,RAM_b,index_size,name_size,size_b,time_%,RAM_%,size_%,ops_%
0,0,QUANTIZE,0.093,0.113,4.730,0,0: quantize,110592,0,0,tfl.quantize,55296,1,tfl.quantize,27648,4.698545,1.688031,2.917841,0.278218
1,1,CONV_2D,0.122,0.149,6.257,1,1: conv_2d,3021912,1431432,1,conv2d,80664,2,shufflenetv2tiny/conv2d/Conv2D,53016,6.195426,2.462444,5.595061,7.602283
2,2,MAX_POOL_2D,0.049,0.049,2.040,2,2: max_pool_2d,124416,0,2,max_pooling2d,66840,3,shufflenetv2tiny/max_pooling2d/MaxPool,13824,2.037422,2.040436,1.458920,0.312996
3,3,CONV_2D,0.043,0.050,2.114,3,3: conv_2d,1410048,663552,3,conv2d_1,41472,4,shufflenetv2tiny/conv2d_1/Conv2D,27648,2.079002,1.266023,2.917841,3.547285
4,4,DEPTHWISE_CONV_2D,0.026,0.031,1.288,4,4: depthwise_conv_2d,65664,31104,4,depthwise_conv2d_1,44928,5,shufflenetv2tiny/depthwise_conv2d_1/depthwise,3456,1.288981,1.371525,0.364730,0.165192
5,5,CONV_2D,0.012,0.014,0.579,5,5: conv_2d,352512,165888,5,conv2d_3,38016,6,shufflenetv2tiny/conv2d_3/Conv2D,6912,0.582121,1.160521,0.729460,0.886821
6,6,DEPTHWISE_CONV_2D,0.044,0.053,2.230,6,6: depthwise_conv_2d,131328,62208,6,batch_normalization_14,41472,7,shufflenetv2tiny/batch_normalization_14/FusedB...,6912,2.203742,1.266023,0.729460,0.330384
7,7,CONV_2D,0.016,0.020,0.836,7,7: conv_2d,684288,331776,7,conv2d_2,20736,8,shufflenetv2tiny/conv2d_2/Conv2D,6912,0.831601,0.633011,0.729460,1.721477
8,8,CONCATENATION,0.001,0.003,0.115,8,8: concatenation,0,0,8,concatenate,27648,9,shufflenetv2tiny/concatenate/concat,13824,0.124740,0.844015,1.458920,0.000000
9,9,SHAPE,0.000,0.001,0.024,9,9: shape,0,0,9,tf.compat.v1.shape,13840,10,shufflenetv2tiny/tf.compat.v1.shape/Shape,16,0.041580,0.422496,0.001689,0.000000


In [421]:
### TODO: add parameters per layer
### TODO: create a function for this fig

trace1 = go.Bar(
    x=combined_df["name"],
    y=combined_df["time_%"],
    name="time %",
    marker=dict(
        color='grey'
               )
)
trace2 = go.Scatter(
    x=combined_df["name"],
    y=combined_df['RAM_%'],
    name='RAM %',
    #yaxis='y2',
    marker_symbol='star',
    marker_color="black",
    mode="markers+text",
    #text = "Columns and elements to be displayed"

)

trace3 = go.Scatter(
    x=combined_df["name"],
    y=combined_df['ops_%'],
    name='ops %',
    #yaxis='y2',
    marker_symbol='circle',
    marker_color="blue",
    mode="markers+text",
    #text = "Columns and elements to be displayed"

)

trace4 = go.Scatter(
    x=combined_df["name"],
    y=combined_df['size_%'],
    name='size %',
    #yaxis='y2',
    marker_symbol='cross', 
    marker_color="orange",
    mode="markers+text",
    #text = "Columns and elements to be displayed"

)

fig= make_subplots()
#fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
#fig.add_trace(trace2,secondary_y=True)
fig.add_trace(trace2)
fig.add_trace(trace3)
fig.add_trace(trace4)
fig['layout'].update(height = 600, width = 1200, title ="Percentage per layer based on complete model - shufflenetv2tiny_0.5_96_c3_o2_f24l1024",xaxis=dict(
      tickangle=-90
    ))
fig.show()

In [422]:
fig = px.bar(combined_df, x=combined_df['index'], y=combined_df['avg ms'])
fig.add_scatter(x=combined_df["index"], y=combined_df["RAM_b"])
fig.show()

In [423]:
trace1 = go.Bar(
    x=combined_df["index"],
    y=combined_df["avg ms"],
    name="avg ms",
    marker=dict(
        color='rgb(34,163,192)'
               )
)
trace2 = go.Scatter(
    x=combined_df["index"],
    y=combined_df['RAM_b'],
    name='RAM',
    yaxis='y2',
    marker_symbol='circle',
    mode="markers+text",
    #text = "Columns and elements to be displayed"

)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 1200, title ="Test",xaxis=dict(
      tickangle=-90
    ))
fig.show()

In [424]:

# dict for the dataframes and their names
dfs = {architecture_1 : run_order_df}


fig = go.Figure(layout_title_text="average runtime in micro seconds per layer")
#fig = make_subplots(rows=2, cols=1, start_cell="bottom-left")

for i in dfs:
    fig = fig.add_trace(go.Bar(x = dfs[i]["index"],
                                   y = dfs[i]["avg ms"], 
                                   name = i,
                                   hoverinfo="all"
                                   #hoverinfo=dfs[i]["Name"]
                                   )
                                   )
            
fig.show()